<a href="https://colab.research.google.com/github/ursulacotrina/Project01/blob/main/consolida.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade -q pygsheets

In [ ]:
!pip install Levenshtein


In [ ]:
#import sys
#sys.executable

# 01 Instalación librerías

In [ ]:
import os
import pickle

import warnings
import datetime
from datetime import datetime
import time
from datetime import date
import pandas as pd
import google.auth
from google.colab import auth
auth.authenticate_user()
import pygsheets
credentials, _ = google.auth.default()
gc = pygsheets.client.Client(credentials)

import numpy as np
import seaborn as sns
import Levenshtein  as lv
from itertools import combinations, permutations
import matplotlib.pyplot as plt

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [ ]:
pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")

# 02 Importación modelo PKL

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
'''
file_path = '/content/drive/MyDrive/Estadística/@Proyectos/016. Modelo de predicción de aprobación de proyectos de ley/scripts/weights_model.pkl'  # Replace with your desired file path

with open(file_path, 'rb') as file:
    weight = pickle.load(file)
'''

"\nfile_path = '/content/drive/MyDrive/Estadística/@Proyectos/016. Modelo de predicción de aprobación de proyectos de ley/scripts/weights_model.pkl'  # Replace with your desired file path\n\nwith open(file_path, 'rb') as file:\n    weight = pickle.load(file)\n"

In [ ]:
file_path = '/content/drive/MyDrive/Estadística/@Proyectos/016. Modelo de predicción de aprobación de proyectos de ley/scripts/model.pkl'  # Replace with your desired file path

with open(file_path, 'rb') as file:
    log_model = pickle.load(file)

In [ ]:
#weight

In [ ]:
'''
intercept = weight['intercept']
coefficients = weight['coefficients']
'''

"\nintercept = weight['intercept']\ncoefficients = weight['coefficients']\n"

In [ ]:
#coefficients

In [ ]:
#log_model=LogisticRegression(class_weight="balanced")

In [ ]:
'''
log_model.intercept_ = intercept
log_model.coef_ = coefficients
'''

'\nlog_model.intercept_ = intercept\nlog_model.coef_ = coefficients\n'

In [ ]:
log_model

LogisticRegression(class_weight='balanced')

In [ ]:
classes = log_model.classes_
classes

array([0, 1])

# 02 Definición de funciones

In [ ]:
def open_gsheets(link, sheet_index):
  sh1 = gc.open_by_url(link)
  wks = sh1[sheet_index]
  data_get = pd.DataFrame(wks.get_all_records()) #get values with out colnames (with colnames drop some columns)
  return data_get

# 03 Importación

## 03.01 Importación "output congreso"

In [ ]:
link = 'https://docs.google.com/spreadsheets/d/16vsPetGMBsRQfz-jIYL7JL32euMCYA6O8-qA48L4W8U/edit#gid=0'

In [ ]:
sheet_index = 2
outputcongreso = open_gsheets(link, sheet_index)
outputcongreso.head(4)

,titulo,periodo,legisla,fechapresenta,proponente,sumilla,observaciones,autor,coautor,adherentes,grupoparla,comisiones,ultimoestado,estadoscrap,autorlink,fecha,enlace,contador_total,dias_proceso,semanas_proceso,semanas_proceso_cat,contador_total_cat,max,brecha_fecha,contador2,dias_proceso2,key_autor,key_congresista,nombre,votacion,inicio,fin,grupopartido,bancada,representa,condicion,_merge
0,PROYECTO DE LEY N° 02871/2022-CR,2021-2026,Primera Legislatura Ordinaria 2022,19/08/2022,Congreso,PROPONE PROTEGER AL CONSUMIDOR BANCARIO Y FINA...,- -,SOTO PALACIOS WILSON,"Martínez Talavera, Pedro Edwin\nDoroteo Carbaj...",- -,Acción Popular,Defensa del Consumidor y Organismos Reguladore...,EN COMISIÓN (Seguimientos),yes,https://www.congreso.gob.pe/congresistas2021/W...,11/07/2023 16:34:07,https://wb2server.congreso.gob.pe/spley-portal...,4,259,1.850000,2,1,2023-05-05,67,16,67081,SOTO PALACIOS WILSON,WILSON SOTO PALACIOS,WILSON SOTO PALACIOS,7145,26-jul-2021,27-jul-2026,Acción Popular,ACCION POPULAR,Huancavelica,en Ejercicio,both
1,PROYECTO DE LEY N° 03739/2022-CR,2021-2026,Primera Legislatura Ordinaria 2022,05/12/2022,Congreso,"PROPONE ELIMINAR LA COMISIÓN INTERPLAZA, LA CO...",- -,FLORES ANCACHI JORGE LUIS,"Vergara Mendoza, Elvis Hernán\nDoroteo Carbajo...",- -,Acción Popular,Defensa del Consumidor y Organismos Reguladore...,Dictamen (Seguimientos),yes,https://www.congreso.gob.pe/congresistas2021/J...,11/07/2023 16:34:14,https://wb2server.congreso.gob.pe/spley-portal...,3,186,1.328571,2,1,2023-06-09,32,9,34596,FLORES ANCACHI JORGE LUIS,JORGE LUIS FLORES ANCACHI,JORGE LUIS FLORES ANCACHI,23539,26-jul-2021,27-jul-2026,Acción Popular,ACCION POPULAR,Puno,en Ejercicio,both
2,PROYECTO DE LEY N° 03262/2022-CR,2021-2026,Primera Legislatura Ordinaria 2022,10/10/2022,Congreso,PROPONE MODIFICAR EL CÓDIGO DE PROTECCIÓN Y DE...,- -,UGARTE MAMANI JHAKELINE KATY,"Quiroz Barboza, Segundo Teodomiro\nGutiérrez T...",- -,Bloque Magisterial de Concertación Nacional,Defensa del Consumidor y Organismos Reguladore...,EN COMISIÓN (Seguimientos),yes,https://www.congreso.gob.pe/congresistas2021/J...,11/07/2023 16:34:16,https://wb2server.congreso.gob.pe/spley-portal...,2,3,0.021429,1,1,2022-10-13,271,4,9,UGARTE MAMANI JHAKELINE KATY,JHAKELINE KATY UGARTE MAMANI,JHAKELINE KATY UGARTE MAMANI,127,26-jul-2021,27-jul-2026,Partido Politico Nacional Perú Libre,NO AGRUPADO,Cusco,en Ejercicio,both
3,PROYECTO DE LEY N° 03428/2022-CR,2021-2026,Primera Legislatura Ordinaria 2022,28/10/2022,Congreso,PROPONE PROTEGER LOS DERECHOS DE LOS CONSUMIDO...,- -,BAZÁN NARRO SIGRID TESORO,"Luque Ibarra, Ruth\nCortez Aguirre, Isabel",- -,Cambio Democrático - Juntos por el Perú,Defensa del Consumidor y Organismos Reguladore...,Dictamen (Seguimientos),yes,https://www.congreso.gob.pe/congresistas2021/S...,11/07/2023 16:34:19,https://wb2server.congreso.gob.pe/spley-portal...,3,224,1.600000,2,1,2023-06-09,32,9,50176,BAZÁN NARRO SIGRID TESORO,SIGRID TESORO BAZÁN NARRO,SIGRID TESORO BAZÁN NARRO,44359,26-jul-2021,27-jul-2026,Juntos por el Perú,CAMBIO DEMOCRÁTICO - JUNTOS POR EL PERÚ,Lima,en Ejercicio,both


## 03.02 Importación "Acción Evaluación"

In [ ]:
link = 'https://docs.google.com/spreadsheets/d/1unHBIVHSB_oqrtDQb7bNunzPiC_VT_GDXjzRFyTTyXw/edit#gid=334199697'


In [ ]:
sheet_index = 1
evaluacion = open_gsheets(link, sheet_index)
evaluacion.head(4)

,Nombre grupo,Nombre Corto,Numero,fechapresenta,autor,enlace,dimensión 01: Nivel de incremento de costos,dimensión 02: Afecta Estado de Derecho,dimensión 03: Nivel de aumento de Carga Operativa,dimensión 04: Disminución de ingresos,dimensión 05: Disminución del nivel de competitividad,Posibilidad de Aprobación
0,Comisiones Bancarias,Interplaza,2871,19/08/2022,SOTO PALACIOS WILSON,https://wb2server.congreso.gob.pe/spley-portal...,Alto,Alto,Positivo,Alto,Positivo,Alto
1,Comisiones Bancarias,"Interplaza, transferencia y pago con otra tarjeta",3739,05/12/2022,FLORES ANCACHI JORGE LUIS,https://wb2server.congreso.gob.pe/spley-portal...,Alto,Alto,Alto,,,Alto
2,Comisiones Bancarias,Pago con cualquier tarjeta (débito o crédito),3262,10/10/2022,UGARTE MAMANI JHAKELINE KATY,https://wb2server.congreso.gob.pe/spley-portal...,Alto,Alto,Alto,,,Alto
3,Comisiones Bancarias,Comisión por transferencia,3428,28/10/2022,BAZÁN NARRO SIGRID TESORO,https://wb2server.congreso.gob.pe/spley-portal...,Alto,Alto,Alto,,,Alto


In [ ]:
outputcongreso.columns

Index(['titulo', 'periodo', 'legisla', 'fechapresenta', 'proponente',
       'sumilla', 'observaciones', 'autor', 'coautor', 'adherentes',
       'grupoparla', 'comisiones', 'ultimoestado', 'estadoscrap', 'autorlink',
       'fecha', 'enlace', 'contador_total', 'dias_proceso', 'semanas_proceso',
       'semanas_proceso_cat', 'contador_total_cat', 'max', 'brecha_fecha',
       'contador2', 'dias_proceso2', 'key_autor', 'key_congresista', 'nombre',
       'votacion', 'inicio', 'fin', 'grupopartido', 'bancada', 'representa',
       'condicion', '_merge'],
      dtype='object')

## 03 03 Importación "X_consolidado"

In [ ]:
file_path = '/content/drive/MyDrive/Estadística/@Proyectos/016. Modelo de predicción de aprobación de proyectos de ley/scripts/X_consolidado.pkl'  # Replace with your desired file path

with open(file_path, 'rb') as file:
    X_consolidado = pickle.load(file)

In [ ]:
X_consolidado

,contador_total,dias_proceso,votacion,semanas_proceso_cat,autor_ACUÑA PERALTA SEGUNDO HÉCTOR,autor_AGUINAGA RECUENCO ALEJANDRO AURELIO,autor_AGÜERO GUTIÉRREZ MARÍA ANTONIETA,autor_ALCARRAZ AGÜERO YOREL KIRA,autor_ALEGRÍA GARCÍA ARTURO,autor_ALVA PRIETO MARÍA DEL CARMEN,autor_ALVA ROJAS CARLOS ENRIQUE,autor_AMURUZ DULANTO YESSICA ROSSELLI,autor_ANDERSON RAMÍREZ CARLOS ANTONIO,autor_ARAGÓN CARREÑO LUIS ÁNGEL,autor_ARRIOLA TUEROS JOSÉ ALBERTO,autor_AZURÍN LOAYZA ALFREDO,autor_BALCÁZAR ZELADA JOSÉ MARÍA,autor_BARBARÁN REYES ROSANGELLA ANDREA,autor_BAZÁN CALDERÓN DIEGO ALONSO FERNANDO,autor_BAZÁN NARRO SIGRID TESORO,autor_BELLIDO UGARTE GUIDO,autor_BERMEJO ROJAS GUILLERMO,autor_BURGOS OLIVEROS JUAN BARTOLOMÉ,autor_BUSTAMANTE DONAYRE CARLOS ERNESTO,autor_CALLE LOBATÓN DIGNA,autor_CAMONES SORIANO LADY MERCEDES,autor_CASTILLO RIVAS EDUARDO ENRIQUE,autor_CAVERO ALVA ALEJANDRO ENRIQUE,autor_CERRÓN ROJAS WALDEMAR JOSÉ,autor_CHACÓN TRUJILLO NILZA MERLY,autor_CHIABRA LEÓN ROBERTO ENRIQUE,autor_CHIRINOS VENEGAS PATRICIA ROSA,autor_CHÁVEZ CHINO BETSSY BETZABET,autor_CICCIA VÁSQUEZ MIGUEL ÁNGEL,autor_COAYLA JUÁREZ JORGE SAMUEL,autor_CORDERO JON TAY LUIS GUSTAVO,autor_CORDERO JON TAY MARÍA DEL PILAR,autor_CORTEZ AGUIRRE ISABEL,autor_CRUZ MAMANI FLAVIO,autor_CUETO ASERVI JOSÉ ERNESTO,autor_CUTIPA CCAMA VÍCTOR RAÚL,autor_CÓRDOVA LOBATÓN MARÍA JESSICA,autor_DOROTEO CARBAJO RAÚL FELIPE,autor_DÁVILA ATANACIO PASIÓN NEOMÍAS,autor_DÍAZ MONAGO FREDDY ROLAND,autor_ECHAÍZ DE NÚÑEZ IZAGA GLADYS MARGOT,autor_ECHEVERRÍA RODRÍGUEZ HAMLET,autor_ELERA GARCÍA WILMAR ALBERTO,autor_ELÍAS ÁVALOS JOSÉ LUIS,autor_ESPINOZA VARGAS JHAEC DARWIN,autor_FLORES ANCACHI JORGE LUIS,autor_FLORES RAMÍREZ ALEX RANDU,autor_FLORES RUIZ VÍCTOR SEFERINO,autor_GARCÍA CORREA IDELSO MANUEL,autor_GONZA CASTILLO AMÉRICO,autor_GONZALES DELGADO DIANA CAROLINA,autor_GUERRA GARCÍA CAMPOS HERNANDO,autor_GUTIÉRREZ TICONA PAUL SILVIO,autor_HEIDINGER BALLESTEROS NELCY LIDIA,autor_HERRERA MAMANI FERNANDO MARIO,autor_HERRERA MEDINA NOELIA ROSSVITH,autor_HUAMÁN CORONADO RAÚL,autor_INFANTES CASTAÑEDA MERY ELIANA,autor_JERÍ ORÉ JOSÉ ENRIQUE,autor_JIMÉNEZ HEREDIA DAVID JULIO,autor_JULÓN IRIGOÍN ELVA EDHIT,autor_JUÁREZ CALLE HEIDY LISBETH,autor_JUÁREZ GALLEGOS CARMEN PATRICIA,autor_JÁUREGUI MARTÍNEZ DE AGUAYO MARÍA DE LOS MILAGROS JACKELINE,autor_KAMICHE MORANTE LUIS ROBERTO,autor_LIMACHI QUISPE NIEVES ESMERALDA,autor_LIZARZABURU LIZARZABURU JUAN CARLOS MARTIN,autor_LUNA GÁLVEZ JOSÉ LEÓN,autor_LUQUE IBARRA RUTH,autor_LÓPEZ MORALES JENY LUZ,autor_LÓPEZ UREÑA ILICH FREDY,autor_MARTICORENA MENDOZA JORGE ALFONSO,autor_MARTÍNEZ TALAVERA PEDRO EDWIN,autor_MEDINA HERMOSILLA ELIZABETH SARA,autor_MEDINA MINAYA ESDRAS RICARDO,autor_MONTALVO CUBAS SEGUNDO TORIBIO,autor_MONTEZA FACHO SILVIA MARÍA,autor_MONTOYA MANRIQUE JORGE CARLOS,autor_MORANTE FIGARI JORGE ALBERTO,autor_MORI CELIS JUAN CARLOS,autor_MOYANO DELGADO MARTHA LUPE,autor_MUÑANTE BARRIOS ALEJANDRO,autor_MÁLAGA TRILLO GEORGE EDWARD,autor_OBANDO MORGAN AURISTELA ANA,autor_OLIVOS MARTÍNEZ VIVIAN,autor_PABLO MEDINA FLOR AIDEE,autor_PADILLA ROMERO JAVIER ROMMEL,autor_PALACIOS HUAMÁN MARGOT,autor_PAREDES CASTRO FRANCIS JHASMINA,autor_PAREDES FONSECA KAROL IVETT,autor_PAREDES GONZALES ALEX ANTONIO,autor_PAREDES PIQUÉ SUSEL ANA MARÍA,autor_PARIONA SINCHE ALFREDO,autor_PAZO NUNURA JOSE BERNARDO,autor_PICÓN QUEDO LUIS RAÚL,autor_PORTALATINO ÁVALOS KELLY ROXANA,autor_PORTERO LÓPEZ HILDA MARLENY,autor_QUIROZ BARBOZA SEGUNDO TEODOMIRO,autor_QUISPE MAMANI WILSON RUSBEL,autor_QUITO SARMIENTO BERNARDO JAIME,autor_RAMÍREZ GARCÍA TANIA ESTEFANY,autor_REVILLA VILLANUEVA CÉSAR MANUEL,autor_REYES CAM ABEL AUGUSTO,autor_REYMUNDO MERCADO EDGARD CORNELIO,autor_RIVAS CHACARA JANET MILAGROS,autor_ROBLES ARAUJO SILVANA EMPERATRIZ,autor_RUÍZ RODRÍGUEZ MAGALY ROSMERY,autor_SAAVEDRA CASTERNOQUE HITLER,autor_SALHUANA CAVIDES EDUARDO,autor_SOTO PALACIOS WILSON,autor_SOTO REYES ALEJANDRO,autor_SÁNCHEZ PALOMINO ROBERTO HELBERT,autor_TACURI VALDIVIA GERMÁN ADOLFO,autor_TAIPE CORONADO M

## 03 04 Importación "append_output"

In [ ]:
link = 'https://docs.google.com/spreadsheets/d/1K63e7VtoTFWyccsQe276_ovoGM7249lYYFC6IL_T_VU/edit#gid=2105803144'

In [ ]:
sheet_index = 1
append_output = open_gsheets(link, sheet_index)
append_output.head(4)

,Nombre grupo,Nombre Corto,Numero,fechapresenta,autor,enlace,dimensión 01: Nivel de incremento de costos,dimensión 02: Afecta Estado de Derecho,dimesión 03: Nivel de aumento de Carga Operativa,dimensión 04: Disminución de ingresos,dimensión 05: Disminución del nivel de competitividad,impacto,Posibilidad de Aprobación,ultimoestado,brecha_fecha,label_pred_prob,prob_aprob,indicador_final,indicador_final_nivel,fecha_scrapping,coautor,grupoparla,comisiones
0,Comisiones Bancarias,Interplaza,2871,19/08/2022,SOTO PALACIOS WILSON,https://wb2server.congreso.gob.pe/spley-portal...,3,3,0,3,0,2,Alto,EN COMISIÓN (Seguimientos),52,0.344312,0.440486,0.880972,ALTO,27-06-2023,,,
1,Comisiones Bancarias,"Interplaza, transferencia y pago con otra tarjeta",3739,05/12/2022,FLORES ANCACHI JORGE LUIS,https://wb2server.congreso.gob.pe/spley-portal...,3,3,3,,,3,Alto,Dictamen (Seguimientos),17,0.110204,0.206378,0.619135,ALTO,27-06-2023,,,
2,Comisiones Bancarias,Pago con cualquier tarjeta (débito o crédito),3262,10/10/2022,UGARTE MAMANI JHAKELINE KATY,https://wb2server.congreso.gob.pe/spley-portal...,3,3,3,,,3,Alto,EN COMISIÓN (Seguimientos),256,0.107684,0.203858,0.611573,ALTO,27-06-2023,,,
3,Comisiones Bancarias,Comisión por transferencia,3428,28/10/2022,BAZÁN NARRO SIGRID TESORO,https://wb2server.congreso.gob.pe/spley-portal...,3,3,3,,,3,Alto,Dictamen (Seguimientos),17,0.098295,0.194469,0.583407,MEDIO,27-06-2023,,,


## 03 05 Importación "scrapping_seguimiento"

In [ ]:
link = 'https://docs.google.com/spreadsheets/d/16vsPetGMBsRQfz-jIYL7JL32euMCYA6O8-qA48L4W8U/edit#gid=2035949973'

sheet_index = 3
scrapping_seguimiento = open_gsheets(link, sheet_index)
scrapping_seguimiento.head(4)

,FECHA,ESTADO PROCESAL,COMISIÓN,DETALLE,ADJUNTOS,enlace,contador,min,max,dias_proceso,contador_total,semanas_proceso,semanas_proceso_cat,contador_total_cat,brecha_fecha,contador2,dias_proceso2
0,2022-08-19,PRESENTADO,,LEY QUE PROTEGE AL CONSUMIDOR BANCARIO Y FINAN...,AdjuntosAdjuntos,https://wb2server.congreso.gob.pe/spley-portal...,1,2022-08-19,2023-05-05,259,4,1.85,2,1,67,16,67081
1,2022-08-24,EN COMISIÓN,Defensa del Consumidor y Organismos Reguladore...,,AdjuntosAdjuntos,https://wb2server.congreso.gob.pe/spley-portal...,2,2022-08-19,2023-05-05,259,4,1.85,2,1,67,16,67081
2,2023-04-27,Dictamen,Defensa del Consumidor y Organismos Reguladore...,EN MAYORÍA - FÓRMULA SUSTITUTORIA - LEY QUE MO...,AdjuntosAdjuntos,https://wb2server.congreso.gob.pe/spley-portal...,3,2022-08-19,2023-05-05,259,4,1.85,2,1,67,16,67081
3,2023-05-05,EN COMISIÓN,"Economía, Banca, Finanzas e Inteligencia Finan...","EL CONSEJO DIRECTIVO, EN SU SESIÓN DEL 2 DE MA...",AdjuntosAdjuntos,https://wb2server.congreso.gob.pe/spley-portal...,4,2022-08-19,2023-05-05,259,4,1.85,2,1,67,16,67081


# 05 Aplicación del modelo

In [ ]:
# Definición variable "Y" del modelo

In [ ]:
outputcongreso['target'] = np.where((outputcongreso['ultimoestado']=='Autógrafa (Seguimientos)') |
 (outputcongreso['ultimoestado']=='Publicada en el Diario Oficial El Peruano (Seguimientos)'), 1, 0)

In [ ]:
outputcongreso.ultimoestado.value_counts()

EN COMISIÓN (Seguimientos)    7
Dictamen (Seguimientos)       2
Name: ultimoestado, dtype: int64

In [ ]:
#df21 = outputcongreso[['autor','grupoparla', 'comisiones', 'contador_total', 'dias_proceso','votacion','representa', 'condicion', 'target']]

df21_base = outputcongreso[['autor','grupoparla', 'comisiones', 'contador_total', 'dias_proceso','votacion','representa', 'condicion', 'target','contador2','dias_proceso2','semanas_proceso_cat','contador_total_cat']]

In [ ]:
df21_base.head()

,autor,grupoparla,comisiones,contador_total,dias_proceso,votacion,representa,condicion,target,contador2,dias_proceso2,semanas_proceso_cat,contador_total_cat
0,SOTO PALACIOS WILSON,Acción Popular,Defensa del Consumidor y Organismos Reguladore...,4,259,7145,Huancavelica,en Ejercicio,0,16,67081,2,1
1,FLORES ANCACHI JORGE LUIS,Acción Popular,Defensa del Consumidor y Organismos Reguladore...,3,186,23539,Puno,en Ejercicio,0,9,34596,2,1
2,UGARTE MAMANI JHAKELINE KATY,Bloque Magisterial de Concertación Nacional,Defensa del Consumidor y Organismos Reguladore...,2,3,127,Cusco,en Ejercicio,0,4,9,1,1
3,BAZÁN NARRO SIGRID TESORO,Cambio Democrático - Juntos por el Perú,Defensa del Consumidor y Organismos Reguladore...,3,224,44359,Lima,en Ejercicio,0,9,50176,2,1
4,CALLE LOBATÓN DIGNA,Podemos Perú,"Economía, Banca, Finanzas e Inteligencia Finan...",2,1,54282,Lima,en Ejercicio,0,4,1,1,1


In [ ]:
oenc=OneHotEncoder(drop='first') #evitar multicolinealidad
multiple_enc=oenc.fit_transform(df21_base[['autor','grupoparla', 'comisiones','representa', 'condicion']])# onehotencoding on 3 features

multiple_enc=multiple_enc.toarray()

#oenc.get_feature_names() use this method to get column names
multiple_enc=pd.DataFrame(multiple_enc,columns=oenc.get_feature_names_out())

df21enc_base=pd.concat([df21_base[['target','contador_total','dias_proceso','votacion','contador2','dias_proceso2','semanas_proceso_cat','contador_total_cat']],multiple_enc],axis=1) # append to original dataframe

In [ ]:
df21enc_base

,target,contador_total,dias_proceso,votacion,contador2,dias_proceso2,semanas_proceso_cat,contador_total_cat,autor_BAZÁN NARRO SIGRID TESORO,autor_CALLE LOBATÓN DIGNA,autor_FLORES ANCACHI JORGE LUIS,autor_OLIVOS MARTÍNEZ VIVIAN,autor_SOTO PALACIOS WILSON,autor_UGARTE MAMANI JHAKELINE KATY,autor_YARROW LUMBRERAS NORMA MARTINA,grupoparla_Avanza País - Partido de Integración Social,grupoparla_Bloque Magisterial de Concertación Nacional,grupoparla_Cambio Democrático - Juntos por el Perú,grupoparla_Fuerza Popular,grupoparla_Podemos Perú,"comisiones_Economía, Banca, Finanzas e Inteligencia Financiera",representa_Huancavelica,representa_Lima,representa_Lima Provincias,representa_Puno,representa_San Martín
0,0,4,259,7145,16,67081,2,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0,3,186,23539,9,34596,2,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0,2,3,127,4,9,1,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,3,224,44359,9,50176,2,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0,2,1,54282,4,1,1,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
5,0,2,1,54282,4,1,1,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
6,0,2,1,2443,4,1,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
7,0,2,1,7785,4,1,1,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
8,0,2,1,51103,4,1,1,1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [ ]:
df21enc_base = pd.concat([X_consolidado,df21enc_base], axis = 0).reset_index()
df21enc_base = df21enc_base.drop ([0],axis=0).reset_index()

In [ ]:
df21enc_base.head()

,level_0,index,contador_total,dias_proceso,votacion,semanas_proceso_cat,autor_ACUÑA PERALTA SEGUNDO HÉCTOR,autor_AGUINAGA RECUENCO ALEJANDRO AURELIO,autor_AGÜERO GUTIÉRREZ MARÍA ANTONIETA,autor_ALCARRAZ AGÜERO YOREL KIRA,autor_ALEGRÍA GARCÍA ARTURO,autor_ALVA PRIETO MARÍA DEL CARMEN,autor_ALVA ROJAS CARLOS ENRIQUE,autor_AMURUZ DULANTO YESSICA ROSSELLI,autor_ANDERSON RAMÍREZ CARLOS ANTONIO,autor_ARAGÓN CARREÑO LUIS ÁNGEL,autor_ARRIOLA TUEROS JOSÉ ALBERTO,autor_AZURÍN LOAYZA ALFREDO,autor_BALCÁZAR ZELADA JOSÉ MARÍA,autor_BARBARÁN REYES ROSANGELLA ANDREA,autor_BAZÁN CALDERÓN DIEGO ALONSO FERNANDO,autor_BAZÁN NARRO SIGRID TESORO,autor_BELLIDO UGARTE GUIDO,autor_BERMEJO ROJAS GUILLERMO,autor_BURGOS OLIVEROS JUAN BARTOLOMÉ,autor_BUSTAMANTE DONAYRE CARLOS ERNESTO,autor_CALLE LOBATÓN DIGNA,autor_CAMONES SORIANO LADY MERCEDES,autor_CASTILLO RIVAS EDUARDO ENRIQUE,autor_CAVERO ALVA ALEJANDRO ENRIQUE,autor_CERRÓN ROJAS WALDEMAR JOSÉ,autor_CHACÓN TRUJILLO NILZA MERLY,autor_CHIABRA LEÓN ROBERTO ENRIQUE,autor_CHIRINOS VENEGAS PATRICIA ROSA,autor_CHÁVEZ CHINO BETSSY BETZABET,autor_CICCIA VÁSQUEZ MIGUEL ÁNGEL,autor_COAYLA JUÁREZ JORGE SAMUEL,autor_CORDERO JON TAY LUIS GUSTAVO,autor_CORDERO JON TAY MARÍA DEL PILAR,autor_CORTEZ AGUIRRE ISABEL,autor_CRUZ MAMANI FLAVIO,autor_CUETO ASERVI JOSÉ ERNESTO,autor_CUTIPA CCAMA VÍCTOR RAÚL,autor_CÓRDOVA LOBATÓN MARÍA JESSICA,autor_DOROTEO CARBAJO RAÚL FELIPE,autor_DÁVILA ATANACIO PASIÓN NEOMÍAS,autor_DÍAZ MONAGO FREDDY ROLAND,autor_ECHAÍZ DE NÚÑEZ IZAGA GLADYS MARGOT,autor_ECHEVERRÍA RODRÍGUEZ HAMLET,autor_ELERA GARCÍA WILMAR ALBERTO,autor_ELÍAS ÁVALOS JOSÉ LUIS,autor_ESPINOZA VARGAS JHAEC DARWIN,autor_FLORES ANCACHI JORGE LUIS,autor_FLORES RAMÍREZ ALEX RANDU,autor_FLORES RUIZ VÍCTOR SEFERINO,autor_GARCÍA CORREA IDELSO MANUEL,autor_GONZA CASTILLO AMÉRICO,autor_GONZALES DELGADO DIANA CAROLINA,autor_GUERRA GARCÍA CAMPOS HERNANDO,autor_GUTIÉRREZ TICONA PAUL SILVIO,autor_HEIDINGER BALLESTEROS NELCY LIDIA,autor_HERRERA MAMANI FERNANDO MARIO,autor_HERRERA MEDINA NOELIA ROSSVITH,autor_HUAMÁN CORONADO RAÚL,autor_INFANTES CASTAÑEDA MERY ELIANA,autor_JERÍ ORÉ JOSÉ ENRIQUE,autor_JIMÉNEZ HEREDIA DAVID JULIO,autor_JULÓN IRIGOÍN ELVA EDHIT,autor_JUÁREZ CALLE HEIDY LISBETH,autor_JUÁREZ GALLEGOS CARMEN PATRICIA,autor_JÁUREGUI MARTÍNEZ DE AGUAYO MARÍA DE LOS MILAGROS JACKELINE,autor_KAMICHE MORANTE LUIS ROBERTO,autor_LIMACHI QUISPE NIEVES ESMERALDA,autor_LIZARZABURU LIZARZABURU JUAN CARLOS MARTIN,autor_LUNA GÁLVEZ JOSÉ LEÓN,autor_LUQUE IBARRA RUTH,autor_LÓPEZ MORALES JENY LUZ,autor_LÓPEZ UREÑA ILICH FREDY,autor_MARTICORENA MENDOZA JORGE ALFONSO,autor_MARTÍNEZ TALAVERA PEDRO EDWIN,autor_MEDINA HERMOSILLA ELIZABETH SARA,autor_MEDINA MINAYA ESDRAS RICARDO,autor_MONTALVO CUBAS SEGUNDO TORIBIO,autor_MONTEZA FACHO SILVIA MARÍA,autor_MONTOYA MANRIQUE JORGE CARLOS,autor_MORANTE FIGARI JORGE ALBERTO,autor_MORI CELIS JUAN CARLOS,autor_MOYANO DELGADO MARTHA LUPE,autor_MUÑANTE BARRIOS ALEJANDRO,autor_MÁLAGA TRILLO GEORGE EDWARD,autor_OBANDO MORGAN AURISTELA ANA,autor_OLIVOS MARTÍNEZ VIVIAN,autor_PABLO MEDINA FLOR AIDEE,autor_PADILLA ROMERO JAVIER ROMMEL,autor_PALACIOS HUAMÁN MARGOT,autor_PAREDES CASTRO FRANCIS JHASMINA,autor_PAREDES FONSECA KAROL IVETT,autor_PAREDES GONZALES ALEX ANTONIO,autor_PAREDES PIQUÉ SUSEL ANA MARÍA,autor_PARIONA SINCHE ALFREDO,autor_PAZO NUNURA JOSE BERNARDO,autor_PICÓN QUEDO LUIS RAÚL,autor_PORTALATINO ÁVALOS KELLY ROXANA,autor_PORTERO LÓPEZ HILDA MARLENY,autor_QUIROZ BARBOZA SEGUNDO TEODOMIRO,autor_QUISPE MAMANI WILSON RUSBEL,autor_QUITO SARMIENTO BERNARDO JAIME,autor_RAMÍREZ GARCÍA TANIA ESTEFANY,autor_REVILLA VILLANUEVA CÉSAR MANUEL,autor_REYES CAM ABEL AUGUSTO,autor_REYMUNDO MERCADO EDGARD CORNELIO,autor_RIVAS CHACARA JANET MILAGROS,autor_ROBLES ARAUJO SILVANA EMPERATRIZ,autor_RUÍZ RODRÍGUEZ MAGALY ROSMERY,autor_SAAVEDRA CASTERNOQUE HITLER,autor_SALHUANA CAVIDES EDUARDO,autor_SOTO PALACIOS WILSON,autor_SOTO REYES ALEJANDRO,autor_SÁNCHEZ PALOMINO ROBERTO HELBERT,autor_TACURI VALDIVIA GERMÁN ADOLFO,autor_TA

In [ ]:
df21enc_base = df21enc_base.drop(['level_0','index'], axis = 1)

In [ ]:
df21enc_base.head()

,contador_total,dias_proceso,votacion,semanas_proceso_cat,autor_ACUÑA PERALTA SEGUNDO HÉCTOR,autor_AGUINAGA RECUENCO ALEJANDRO AURELIO,autor_AGÜERO GUTIÉRREZ MARÍA ANTONIETA,autor_ALCARRAZ AGÜERO YOREL KIRA,autor_ALEGRÍA GARCÍA ARTURO,autor_ALVA PRIETO MARÍA DEL CARMEN,autor_ALVA ROJAS CARLOS ENRIQUE,autor_AMURUZ DULANTO YESSICA ROSSELLI,autor_ANDERSON RAMÍREZ CARLOS ANTONIO,autor_ARAGÓN CARREÑO LUIS ÁNGEL,autor_ARRIOLA TUEROS JOSÉ ALBERTO,autor_AZURÍN LOAYZA ALFREDO,autor_BALCÁZAR ZELADA JOSÉ MARÍA,autor_BARBARÁN REYES ROSANGELLA ANDREA,autor_BAZÁN CALDERÓN DIEGO ALONSO FERNANDO,autor_BAZÁN NARRO SIGRID TESORO,autor_BELLIDO UGARTE GUIDO,autor_BERMEJO ROJAS GUILLERMO,autor_BURGOS OLIVEROS JUAN BARTOLOMÉ,autor_BUSTAMANTE DONAYRE CARLOS ERNESTO,autor_CALLE LOBATÓN DIGNA,autor_CAMONES SORIANO LADY MERCEDES,autor_CASTILLO RIVAS EDUARDO ENRIQUE,autor_CAVERO ALVA ALEJANDRO ENRIQUE,autor_CERRÓN ROJAS WALDEMAR JOSÉ,autor_CHACÓN TRUJILLO NILZA MERLY,autor_CHIABRA LEÓN ROBERTO ENRIQUE,autor_CHIRINOS VENEGAS PATRICIA ROSA,autor_CHÁVEZ CHINO BETSSY BETZABET,autor_CICCIA VÁSQUEZ MIGUEL ÁNGEL,autor_COAYLA JUÁREZ JORGE SAMUEL,autor_CORDERO JON TAY LUIS GUSTAVO,autor_CORDERO JON TAY MARÍA DEL PILAR,autor_CORTEZ AGUIRRE ISABEL,autor_CRUZ MAMANI FLAVIO,autor_CUETO ASERVI JOSÉ ERNESTO,autor_CUTIPA CCAMA VÍCTOR RAÚL,autor_CÓRDOVA LOBATÓN MARÍA JESSICA,autor_DOROTEO CARBAJO RAÚL FELIPE,autor_DÁVILA ATANACIO PASIÓN NEOMÍAS,autor_DÍAZ MONAGO FREDDY ROLAND,autor_ECHAÍZ DE NÚÑEZ IZAGA GLADYS MARGOT,autor_ECHEVERRÍA RODRÍGUEZ HAMLET,autor_ELERA GARCÍA WILMAR ALBERTO,autor_ELÍAS ÁVALOS JOSÉ LUIS,autor_ESPINOZA VARGAS JHAEC DARWIN,autor_FLORES ANCACHI JORGE LUIS,autor_FLORES RAMÍREZ ALEX RANDU,autor_FLORES RUIZ VÍCTOR SEFERINO,autor_GARCÍA CORREA IDELSO MANUEL,autor_GONZA CASTILLO AMÉRICO,autor_GONZALES DELGADO DIANA CAROLINA,autor_GUERRA GARCÍA CAMPOS HERNANDO,autor_GUTIÉRREZ TICONA PAUL SILVIO,autor_HEIDINGER BALLESTEROS NELCY LIDIA,autor_HERRERA MAMANI FERNANDO MARIO,autor_HERRERA MEDINA NOELIA ROSSVITH,autor_HUAMÁN CORONADO RAÚL,autor_INFANTES CASTAÑEDA MERY ELIANA,autor_JERÍ ORÉ JOSÉ ENRIQUE,autor_JIMÉNEZ HEREDIA DAVID JULIO,autor_JULÓN IRIGOÍN ELVA EDHIT,autor_JUÁREZ CALLE HEIDY LISBETH,autor_JUÁREZ GALLEGOS CARMEN PATRICIA,autor_JÁUREGUI MARTÍNEZ DE AGUAYO MARÍA DE LOS MILAGROS JACKELINE,autor_KAMICHE MORANTE LUIS ROBERTO,autor_LIMACHI QUISPE NIEVES ESMERALDA,autor_LIZARZABURU LIZARZABURU JUAN CARLOS MARTIN,autor_LUNA GÁLVEZ JOSÉ LEÓN,autor_LUQUE IBARRA RUTH,autor_LÓPEZ MORALES JENY LUZ,autor_LÓPEZ UREÑA ILICH FREDY,autor_MARTICORENA MENDOZA JORGE ALFONSO,autor_MARTÍNEZ TALAVERA PEDRO EDWIN,autor_MEDINA HERMOSILLA ELIZABETH SARA,autor_MEDINA MINAYA ESDRAS RICARDO,autor_MONTALVO CUBAS SEGUNDO TORIBIO,autor_MONTEZA FACHO SILVIA MARÍA,autor_MONTOYA MANRIQUE JORGE CARLOS,autor_MORANTE FIGARI JORGE ALBERTO,autor_MORI CELIS JUAN CARLOS,autor_MOYANO DELGADO MARTHA LUPE,autor_MUÑANTE BARRIOS ALEJANDRO,autor_MÁLAGA TRILLO GEORGE EDWARD,autor_OBANDO MORGAN AURISTELA ANA,autor_OLIVOS MARTÍNEZ VIVIAN,autor_PABLO MEDINA FLOR AIDEE,autor_PADILLA ROMERO JAVIER ROMMEL,autor_PALACIOS HUAMÁN MARGOT,autor_PAREDES CASTRO FRANCIS JHASMINA,autor_PAREDES FONSECA KAROL IVETT,autor_PAREDES GONZALES ALEX ANTONIO,autor_PAREDES PIQUÉ SUSEL ANA MARÍA,autor_PARIONA SINCHE ALFREDO,autor_PAZO NUNURA JOSE BERNARDO,autor_PICÓN QUEDO LUIS RAÚL,autor_PORTALATINO ÁVALOS KELLY ROXANA,autor_PORTERO LÓPEZ HILDA MARLENY,autor_QUIROZ BARBOZA SEGUNDO TEODOMIRO,autor_QUISPE MAMANI WILSON RUSBEL,autor_QUITO SARMIENTO BERNARDO JAIME,autor_RAMÍREZ GARCÍA TANIA ESTEFANY,autor_REVILLA VILLANUEVA CÉSAR MANUEL,autor_REYES CAM ABEL AUGUSTO,autor_REYMUNDO MERCADO EDGARD CORNELIO,autor_RIVAS CHACARA JANET MILAGROS,autor_ROBLES ARAUJO SILVANA EMPERATRIZ,autor_RUÍZ RODRÍGUEZ MAGALY ROSMERY,autor_SAAVEDRA CASTERNOQUE HITLER,autor_SALHUANA CAVIDES EDUARDO,autor_SOTO PALACIOS WILSON,autor_SOTO REYES ALEJANDRO,autor_SÁNCHEZ PALOMINO ROBERTO HELBERT,autor_TACURI VALDIVIA GERMÁN ADOLFO,autor_TAIPE CORONADO M

In [ ]:
df21enc_base.replace(np.nan, 0.0, inplace = True)

In [ ]:
df21enc_base.columns

Index(['contador_total', 'dias_proceso', 'votacion', 'semanas_proceso_cat',
       'autor_ACUÑA PERALTA SEGUNDO HÉCTOR',
       'autor_AGUINAGA RECUENCO ALEJANDRO AURELIO',
       'autor_AGÜERO GUTIÉRREZ MARÍA ANTONIETA',
       'autor_ALCARRAZ AGÜERO YOREL KIRA', 'autor_ALEGRÍA GARCÍA ARTURO',
       'autor_ALVA PRIETO MARÍA DEL CARMEN',
       ...
       'representa_Ucayali', 'representa_Áncash', 'condicion_Fallecido',
       'condicion_Suspendido',
       'condicion_Suspendido por Acusación Constitucional',
       'condicion_en Ejercicio', 'target', 'contador2', 'dias_proceso2',
       'contador_total_cat'],
      dtype='object', length=213)

In [ ]:
#Modelo 3: Variable categorica de semanas prcoeso
df21enc_columns = df21enc_base.columns.tolist()
df21enc_columns.remove('contador_total_cat')
df21enc_columns.remove('contador2')
df21enc_columns.remove('dias_proceso2')
df21enc_columns.remove('target')
df21enc_columns3 = df21enc_columns


In [ ]:
X_subset_mod3 = df21enc_base[df21enc_columns3]
X_subset_mod3

,contador_total,dias_proceso,votacion,semanas_proceso_cat,autor_ACUÑA PERALTA SEGUNDO HÉCTOR,autor_AGUINAGA RECUENCO ALEJANDRO AURELIO,autor_AGÜERO GUTIÉRREZ MARÍA ANTONIETA,autor_ALCARRAZ AGÜERO YOREL KIRA,autor_ALEGRÍA GARCÍA ARTURO,autor_ALVA PRIETO MARÍA DEL CARMEN,autor_ALVA ROJAS CARLOS ENRIQUE,autor_AMURUZ DULANTO YESSICA ROSSELLI,autor_ANDERSON RAMÍREZ CARLOS ANTONIO,autor_ARAGÓN CARREÑO LUIS ÁNGEL,autor_ARRIOLA TUEROS JOSÉ ALBERTO,autor_AZURÍN LOAYZA ALFREDO,autor_BALCÁZAR ZELADA JOSÉ MARÍA,autor_BARBARÁN REYES ROSANGELLA ANDREA,autor_BAZÁN CALDERÓN DIEGO ALONSO FERNANDO,autor_BAZÁN NARRO SIGRID TESORO,autor_BELLIDO UGARTE GUIDO,autor_BERMEJO ROJAS GUILLERMO,autor_BURGOS OLIVEROS JUAN BARTOLOMÉ,autor_BUSTAMANTE DONAYRE CARLOS ERNESTO,autor_CALLE LOBATÓN DIGNA,autor_CAMONES SORIANO LADY MERCEDES,autor_CASTILLO RIVAS EDUARDO ENRIQUE,autor_CAVERO ALVA ALEJANDRO ENRIQUE,autor_CERRÓN ROJAS WALDEMAR JOSÉ,autor_CHACÓN TRUJILLO NILZA MERLY,autor_CHIABRA LEÓN ROBERTO ENRIQUE,autor_CHIRINOS VENEGAS PATRICIA ROSA,autor_CHÁVEZ CHINO BETSSY BETZABET,autor_CICCIA VÁSQUEZ MIGUEL ÁNGEL,autor_COAYLA JUÁREZ JORGE SAMUEL,autor_CORDERO JON TAY LUIS GUSTAVO,autor_CORDERO JON TAY MARÍA DEL PILAR,autor_CORTEZ AGUIRRE ISABEL,autor_CRUZ MAMANI FLAVIO,autor_CUETO ASERVI JOSÉ ERNESTO,autor_CUTIPA CCAMA VÍCTOR RAÚL,autor_CÓRDOVA LOBATÓN MARÍA JESSICA,autor_DOROTEO CARBAJO RAÚL FELIPE,autor_DÁVILA ATANACIO PASIÓN NEOMÍAS,autor_DÍAZ MONAGO FREDDY ROLAND,autor_ECHAÍZ DE NÚÑEZ IZAGA GLADYS MARGOT,autor_ECHEVERRÍA RODRÍGUEZ HAMLET,autor_ELERA GARCÍA WILMAR ALBERTO,autor_ELÍAS ÁVALOS JOSÉ LUIS,autor_ESPINOZA VARGAS JHAEC DARWIN,autor_FLORES ANCACHI JORGE LUIS,autor_FLORES RAMÍREZ ALEX RANDU,autor_FLORES RUIZ VÍCTOR SEFERINO,autor_GARCÍA CORREA IDELSO MANUEL,autor_GONZA CASTILLO AMÉRICO,autor_GONZALES DELGADO DIANA CAROLINA,autor_GUERRA GARCÍA CAMPOS HERNANDO,autor_GUTIÉRREZ TICONA PAUL SILVIO,autor_HEIDINGER BALLESTEROS NELCY LIDIA,autor_HERRERA MAMANI FERNANDO MARIO,autor_HERRERA MEDINA NOELIA ROSSVITH,autor_HUAMÁN CORONADO RAÚL,autor_INFANTES CASTAÑEDA MERY ELIANA,autor_JERÍ ORÉ JOSÉ ENRIQUE,autor_JIMÉNEZ HEREDIA DAVID JULIO,autor_JULÓN IRIGOÍN ELVA EDHIT,autor_JUÁREZ CALLE HEIDY LISBETH,autor_JUÁREZ GALLEGOS CARMEN PATRICIA,autor_JÁUREGUI MARTÍNEZ DE AGUAYO MARÍA DE LOS MILAGROS JACKELINE,autor_KAMICHE MORANTE LUIS ROBERTO,autor_LIMACHI QUISPE NIEVES ESMERALDA,autor_LIZARZABURU LIZARZABURU JUAN CARLOS MARTIN,autor_LUNA GÁLVEZ JOSÉ LEÓN,autor_LUQUE IBARRA RUTH,autor_LÓPEZ MORALES JENY LUZ,autor_LÓPEZ UREÑA ILICH FREDY,autor_MARTICORENA MENDOZA JORGE ALFONSO,autor_MARTÍNEZ TALAVERA PEDRO EDWIN,autor_MEDINA HERMOSILLA ELIZABETH SARA,autor_MEDINA MINAYA ESDRAS RICARDO,autor_MONTALVO CUBAS SEGUNDO TORIBIO,autor_MONTEZA FACHO SILVIA MARÍA,autor_MONTOYA MANRIQUE JORGE CARLOS,autor_MORANTE FIGARI JORGE ALBERTO,autor_MORI CELIS JUAN CARLOS,autor_MOYANO DELGADO MARTHA LUPE,autor_MUÑANTE BARRIOS ALEJANDRO,autor_MÁLAGA TRILLO GEORGE EDWARD,autor_OBANDO MORGAN AURISTELA ANA,autor_OLIVOS MARTÍNEZ VIVIAN,autor_PABLO MEDINA FLOR AIDEE,autor_PADILLA ROMERO JAVIER ROMMEL,autor_PALACIOS HUAMÁN MARGOT,autor_PAREDES CASTRO FRANCIS JHASMINA,autor_PAREDES FONSECA KAROL IVETT,autor_PAREDES GONZALES ALEX ANTONIO,autor_PAREDES PIQUÉ SUSEL ANA MARÍA,autor_PARIONA SINCHE ALFREDO,autor_PAZO NUNURA JOSE BERNARDO,autor_PICÓN QUEDO LUIS RAÚL,autor_PORTALATINO ÁVALOS KELLY ROXANA,autor_PORTERO LÓPEZ HILDA MARLENY,autor_QUIROZ BARBOZA SEGUNDO TEODOMIRO,autor_QUISPE MAMANI WILSON RUSBEL,autor_QUITO SARMIENTO BERNARDO JAIME,autor_RAMÍREZ GARCÍA TANIA ESTEFANY,autor_REVILLA VILLANUEVA CÉSAR MANUEL,autor_REYES CAM ABEL AUGUSTO,autor_REYMUNDO MERCADO EDGARD CORNELIO,autor_RIVAS CHACARA JANET MILAGROS,autor_ROBLES ARAUJO SILVANA EMPERATRIZ,autor_RUÍZ RODRÍGUEZ MAGALY ROSMERY,autor_SAAVEDRA CASTERNOQUE HITLER,autor_SALHUANA CAVIDES EDUARDO,autor_SOTO PALACIOS WILSON,autor_SOTO REYES ALEJANDRO,autor_SÁNCHEZ PALOMINO ROBERTO HELBERT,autor_TACURI VALDIVIA GERMÁN ADOLFO,autor_TAIPE CORONADO M

In [ ]:
label_pred = (log_model.predict_proba(X_subset_mod3)[:,1]>0.5).astype(int)
label_pred_prob = log_model.predict_proba(X_subset_mod3)[:,1]

In [ ]:
final1 = pd.DataFrame({'id':X_subset_mod3.index, 'label_pred': label_pred, 'label_pred_prob': label_pred_prob})
final1

,id,label_pred,label_pred_prob
0,0,0,0.344312
1,1,0,0.110204
2,2,0,0.107684
3,3,0,0.098295
4,4,0,0.033485
5,5,0,0.061979
6,6,0,0.042814
7,7,0,0.042878
8,8,0,0.058308


In [ ]:
dff1 = pd.concat([outputcongreso, final1], axis=1)

In [ ]:
dff1

,titulo,periodo,legisla,fechapresenta,proponente,sumilla,observaciones,autor,coautor,adherentes,grupoparla,comisiones,ultimoestado,estadoscrap,autorlink,fecha,enlace,contador_total,dias_proceso,semanas_proceso,semanas_proceso_cat,contador_total_cat,max,brecha_fecha,contador2,dias_proceso2,key_autor,key_congresista,nombre,votacion,inicio,fin,grupopartido,bancada,representa,condicion,_merge,target,id,label_pred,label_pred_prob
0,PROYECTO DE LEY N° 02871/2022-CR,2021-2026,Primera Legislatura Ordinaria 2022,19/08/2022,Congreso,PROPONE PROTEGER AL CONSUMIDOR BANCARIO Y FINA...,- -,SOTO PALACIOS WILSON,"Martínez Talavera, Pedro Edwin\nDoroteo Carbaj...",- -,Acción Popular,Defensa del Consumidor y Organismos Reguladore...,EN COMISIÓN (Seguimientos),yes,https://www.congreso.gob.pe/congresistas2021/W...,11/07/2023 16:34:07,https://wb2server.congreso.gob.pe/spley-portal...,4,259,1.850000,2,1,2023-05-05,67,16,67081,SOTO PALACIOS WILSON,WILSON SOTO PALACIOS,WILSON SOTO PALACIOS,7145,26-jul-2021,27-jul-2026,Acción Popular,ACCION POPULAR,Huancavelica,en Ejercicio,both,0,0,0,0.344312
1,PROYECTO DE LEY N° 03739/2022-CR,2021-2026,Primera Legislatura Ordinaria 2022,05/12/2022,Congreso,"PROPONE ELIMINAR LA COMISIÓN INTERPLAZA, LA CO...",- -,FLORES ANCACHI JORGE LUIS,"Vergara Mendoza, Elvis Hernán\nDoroteo Carbajo...",- -,Acción Popular,Defensa del Consumidor y Organismos Reguladore...,Dictamen (Seguimientos),yes,https://www.congreso.gob.pe/congresistas2021/J...,11/07/2023 16:34:14,https://wb2server.congreso.gob.pe/spley-portal...,3,186,1.328571,2,1,2023-06-09,32,9,34596,FLORES ANCACHI JORGE LUIS,JORGE LUIS FLORES ANCACHI,JORGE LUIS FLORES ANCACHI,23539,26-jul-2021,27-jul-2026,Acción Popular,ACCION POPULAR,Puno,en Ejercicio,both,0,1,0,0.110204
2,PROYECTO DE LEY N° 03262/2022-CR,2021-2026,Primera Legislatura Ordinaria 2022,10/10/2022,Congreso,PROPONE MODIFICAR EL CÓDIGO DE PROTECCIÓN Y DE...,- -,UGARTE MAMANI JHAKELINE KATY,"Quiroz Barboza, Segundo Teodomiro\nGutiérrez T...",- -,Bloque Magisterial de Concertación Nacional,Defensa del Consumidor y Organismos Reguladore...,EN COMISIÓN (Seguimientos),yes,https://www.congreso.gob.pe/congresistas2021/J...,11/07/2023 16:34:16,https://wb2server.congreso.gob.pe/spley-portal...,2,3,0.021429,1,1,2022-10-13,271,4,9,UGARTE MAMANI JHAKELINE KATY,JHAKELINE KATY UGARTE MAMANI,JHAKELINE KATY UGARTE MAMANI,127,26-jul-2021,27-jul-2026,Partido Politico Nacional Perú Libre,NO AGRUPADO,Cusco,en Ejercicio,both,0,2,0,0.107684
3,PROYECTO DE LEY N° 03428/2022-CR,2021-2026,Primera Legislatura Ordinaria 2022,28/10/2022,Congreso,PROPONE PROTEGER LOS DERECHOS DE LOS CONSUMIDO...,- -,BAZÁN NARRO SIGRID TESORO,"Luque Ibarra, Ruth\nCortez Aguirre, Isabel",- -,Cambio Democrático - Juntos por el Perú,Defensa del Consumidor y Organismos Reguladore...,Dictamen (Seguimientos),yes,https://www.congreso.gob.pe/congresistas2021/S...,11/07/2023 16:34:19,https://wb2server.congreso.gob.pe/spley-portal...,3,224,1.600000,2,1,2023-06-09,32,9,50176,BAZÁN NARRO SIGRID TESORO,SIGRID TESORO BAZÁN NARRO,SIGRID TESORO BAZÁN NARRO,44359,26-jul-2021,27-jul-2026,Juntos por el Perú,CAMBIO DEMOCRÁTICO - JUNTOS POR EL PERÚ,Lima,en Ejercicio,both,0,3,0,0.098295
4,PROYECTO DE LEY N° 04359/2022-CR,2021-2026,Segunda Legislatura Ordinaria 2022,02/03/2023,Congreso,PROPONE ELIMINAR LA COMISIÓN POR RETIRO DE DIN...,- -,CALLE LOBATÓN DIGNA,"Luna Gálvez, José León\nElías Ávalos, José Lui...",- -,Podemos Perú,"Economía, Banca, Finanzas e Inteligencia Finan...",EN COMISIÓN (Seguimientos),yes,https://www.congreso.gob.pe/congresistas2021/D...,11/07/2023 16:34:21,https://wb2server.congreso.gob.pe/spley-portal...,2,1,0.007143,1,1,2023-03-03,130,4,1,CALLE LOBATÓN DIGNA,DIGNA CALLE LOBATÓN,DIGNA CALLE LOBATÓN,54282,26-jul-2021,27-jul-2026,Podemos Perú,PODEMOS PERÚ,Lima,en Ejercicio,both,0,4,0,0.033485
5,PROYECTO DE LEY N° 04360/2022-CR,2021-2026,Segunda Legislatura Ordinaria 2022,02/03/2023,Congreso,PROPONE RECONOCER EL DERECHO A LOS CONSUMIDORE...,- -,CALLE LOBATÓN DIGNA,

# 04 Merge

In [ ]:
df=pd.merge(evaluacion, dff1[['ultimoestado','brecha_fecha','enlace','label_pred_prob','coautor','grupoparla','sumilla']], on="enlace", how="inner", indicator=True)
df.head(2)

,Nombre grupo,Nombre Corto,Numero,fechapresenta,autor,enlace,dimensión 01: Nivel de incremento de costos,dimensión 02: Afecta Estado de Derecho,dimensión 03: Nivel de aumento de Carga Operativa,dimensión 04: Disminución de ingresos,dimensión 05: Disminución del nivel de competitividad,Posibilidad de Aprobación,ultimoestado,brecha_fecha,label_pred_prob,coautor,grupoparla,sumilla,_merge
0,Comisiones Bancarias,Interplaza,2871,19/08/2022,SOTO PALACIOS WILSON,https://wb2server.congreso.gob.pe/spley-portal...,Alto,Alto,Positivo,Alto,Positivo,Alto,EN COMISIÓN (Seguimientos),67,0.344312,"Martínez Talavera, Pedro Edwin\nDoroteo Carbaj...",Acción Popular,PROPONE PROTEGER AL CONSUMIDOR BANCARIO Y FINA...,both
1,Comisiones Bancarias,"Interplaza, transferencia y pago con otra tarjeta",3739,05/12/2022,FLORES ANCACHI JORGE LUIS,https://wb2server.congreso.gob.pe/spley-portal...,Alto,Alto,Alto,,,Alto,Dictamen (Seguimientos),32,0.110204,"Vergara Mendoza, Elvis Hernán\nDoroteo Carbajo...",Acción Popular,"PROPONE ELIMINAR LA COMISIÓN INTERPLAZA, LA CO...",both


In [ ]:
df.insert(5, 'coautor', df.pop('coautor'))
df.insert(6, 'grupoparla', df.pop('grupoparla'))
df.insert(7, 'ultimoestado', df.pop('ultimoestado'))
df.insert(8, 'sumilla', df.pop('sumilla'))

In [ ]:
df.head()

,Nombre grupo,Nombre Corto,Numero,fechapresenta,autor,coautor,grupoparla,ultimoestado,sumilla,enlace,dimensión 01: Nivel de incremento de costos,dimensión 02: Afecta Estado de Derecho,dimensión 03: Nivel de aumento de Carga Operativa,dimensión 04: Disminución de ingresos,dimensión 05: Disminución del nivel de competitividad,Posibilidad de Aprobación,brecha_fecha,label_pred_prob,_merge
0,Comisiones Bancarias,Interplaza,2871,19/08/2022,SOTO PALACIOS WILSON,"Martínez Talavera, Pedro Edwin\nDoroteo Carbaj...",Acción Popular,EN COMISIÓN (Seguimientos),PROPONE PROTEGER AL CONSUMIDOR BANCARIO Y FINA...,https://wb2server.congreso.gob.pe/spley-portal...,Alto,Alto,Positivo,Alto,Positivo,Alto,67,0.344312,both
1,Comisiones Bancarias,"Interplaza, transferencia y pago con otra tarjeta",3739,05/12/2022,FLORES ANCACHI JORGE LUIS,"Vergara Mendoza, Elvis Hernán\nDoroteo Carbajo...",Acción Popular,Dictamen (Seguimientos),"PROPONE ELIMINAR LA COMISIÓN INTERPLAZA, LA CO...",https://wb2server.congreso.gob.pe/spley-portal...,Alto,Alto,Alto,,,Alto,32,0.110204,both
2,Comisiones Bancarias,Pago con cualquier tarjeta (débito o crédito),3262,10/10/2022,UGARTE MAMANI JHAKELINE KATY,"Quiroz Barboza, Segundo Teodomiro\nGutiérrez T...",Bloque Magisterial de Concertación Nacional,EN COMISIÓN (Seguimientos),PROPONE MODIFICAR EL CÓDIGO DE PROTECCIÓN Y DE...,https://wb2server.congreso.gob.pe/spley-portal...,Alto,Alto,Alto,,,Alto,271,0.107684,both
3,Comisiones Bancarias,Comisión por transferencia,3428,28/10/2022,BAZÁN NARRO SIGRID TESORO,"Luque Ibarra, Ruth\nCortez Aguirre, Isabel",Cambio Democrático - Juntos por el Perú,Dictamen (Seguimientos),PROPONE PROTEGER LOS DERECHOS DE LOS CONSUMIDO...,https://wb2server.congreso.gob.pe/spley-portal...,Alto,Alto,Alto,,,Alto,32,0.098295,both
4,Comisiones Bancarias,Comisión por depósito y retiro de dinero en ve...,4359,02/03/2023,CALLE LOBATÓN DIGNA,"Luna Gálvez, José León\nElías Ávalos, José Lui...",Podemos Perú,EN COMISIÓN (Seguimientos),PROPONE ELIMINAR LA COMISIÓN POR RETIRO DE DIN...,https://wb2server.congreso.gob.pe/spley-portal...,Alto,Alto,Medio,,,Alto,130,0.033485,both


In [ ]:
df.shape

(9, 19)

# 05 Preparación base

## 05 01. Conversión de categóricas a numéricas "Dimensiones"

In [ ]:
df.columns

Index(['Nombre grupo', 'Nombre Corto', 'Numero', 'fechapresenta', 'autor',
       'coautor', 'grupoparla', 'ultimoestado', 'sumilla', 'enlace',
       'dimensión 01: Nivel de incremento de costos',
       'dimensión 02: Afecta Estado de Derecho',
       'dimensión 03: Nivel de aumento de Carga Operativa',
       'dimensión 04: Disminución de ingresos',
       'dimensión 05: Disminución del nivel de competitividad',
       'Posibilidad de Aprobación', 'brecha_fecha', 'label_pred_prob',
       '_merge'],
      dtype='object')

In [ ]:
# df = df.drop(['dimensión 05: Incrementa el régimen impositivo'], axis=1)

In [ ]:
mapping = {'Positivo': 0, 'Bajo': 1, 'Medio': 2, 'Alto': 3}

df['dimensión 01: Nivel de incremento de costos'] = pd.to_numeric(df['dimensión 01: Nivel de incremento de costos'].replace(mapping))
df['dimensión 02: Afecta Estado de Derecho'] = pd.to_numeric(df['dimensión 02: Afecta Estado de Derecho'].replace(mapping))
df['dimensión 03: Nivel de aumento de Carga Operativa'] = pd.to_numeric(df['dimensión 03: Nivel de aumento de Carga Operativa'].replace(mapping))
df['dimensión 04: Disminución de ingresos'] = pd.to_numeric(df['dimensión 04: Disminución de ingresos'].replace(mapping))
df['dimensión 05: Disminución del nivel de competitividad'] = pd.to_numeric(df['dimensión 05: Disminución del nivel de competitividad'].replace(mapping))

In [ ]:
df['dimensión 02: Afecta Estado de Derecho'].value_counts()

3    6
2    3
Name: dimensión 02: Afecta Estado de Derecho, dtype: int64

In [ ]:
df

,Nombre grupo,Nombre Corto,Numero,fechapresenta,autor,coautor,grupoparla,ultimoestado,sumilla,enlace,dimensión 01: Nivel de incremento de costos,dimensión 02: Afecta Estado de Derecho,dimensión 03: Nivel de aumento de Carga Operativa,dimensión 04: Disminución de ingresos,dimensión 05: Disminución del nivel de competitividad,Posibilidad de Aprobación,brecha_fecha,label_pred_prob,_merge
0,Comisiones Bancarias,Interplaza,2871,19/08/2022,SOTO PALACIOS WILSON,"Martínez Talavera, Pedro Edwin\nDoroteo Carbaj...",Acción Popular,EN COMISIÓN (Seguimientos),PROPONE PROTEGER AL CONSUMIDOR BANCARIO Y FINA...,https://wb2server.congreso.gob.pe/spley-portal...,3,3,0,3.0,0.0,Alto,67,0.344312,both
1,Comisiones Bancarias,"Interplaza, transferencia y pago con otra tarjeta",3739,05/12/2022,FLORES ANCACHI JORGE LUIS,"Vergara Mendoza, Elvis Hernán\nDoroteo Carbajo...",Acción Popular,Dictamen (Seguimientos),"PROPONE ELIMINAR LA COMISIÓN INTERPLAZA, LA CO...",https://wb2server.congreso.gob.pe/spley-portal...,3,3,3,NaN,NaN,Alto,32,0.110204,both
2,Comisiones Bancarias,Pago con cualquier tarjeta (débito o crédito),3262,10/10/2022,UGARTE MAMANI JHAKELINE KATY,"Quiroz Barboza, Segundo Teodomiro\nGutiérrez T...",Bloque Magisterial de Concertación Nacional,EN COMISIÓN (Seguimientos),PROPONE MODIFICAR EL CÓDIGO DE PROTECCIÓN Y DE...,https://wb2server.congreso.gob.pe/spley-portal...,3,3,3,NaN,NaN,Alto,271,0.107684,both
3,Comisiones Bancarias,Comisión por transferencia,3428,28/10/2022,BAZÁN NARRO SIGRID TESORO,"Luque Ibarra, Ruth\nCortez Aguirre, Isabel",Cambio Democrático - Juntos por el Perú,Dictamen (Seguimientos),PROPONE PROTEGER LOS DERECHOS DE LOS CONSUMIDO...,https://wb2server.congreso.gob.pe/spley-portal...,3,3,3,NaN,NaN,Alto,32,0.098295,both
4,Comisiones Bancarias,Comisión por depósito y retiro de dinero en ve...,4359,02/03/2023,CALLE LOBATÓN DIGNA,"Luna Gálvez, José León\nElías Ávalos, José Lui...",Podemos Perú,EN COMISIÓN (Seguimientos),PROPONE ELIMINAR LA COMISIÓN POR RETIRO DE DIN...,https://wb2server.congreso.gob.pe/spley-portal...,3,3,2,NaN,NaN,Alto,130,0.033485,both
5,Comisiones Bancarias,Bloqueo de banca móvil,4360,02/03/2023,CALLE LOBATÓN DIGNA,"Luna Gálvez, José León\nElías Ávalos, José Lui...",Podemos Perú,EN COMISIÓN (Seguimientos),PROPONE RECONOCER EL DERECHO A LOS CONSUMIDORE...,https://wb2server.congreso.gob.pe/spley-portal...,3,2,2,NaN,NaN,Medio,130,0.061979,both
6,Reprogramación de Deudas por Lluvias,Proyecto de Ley 4525 y 4704: Reprogramación de...,4525,21/03/2023,ALEGRÍA GARCÍA ARTURO,"Lizarzaburu Lizarzaburu, Juan Carlos Martin\nG...",Fuerza Popular,EN COMISIÓN (Seguimientos),PROPONE LEY DE CONGELAMIENTO DE DEUDAS DE PERS...,https://wb2server.congreso.gob.pe/spley-portal...,3,2,2,3.0,NaN,Medio,111,0.042814,both
7,Reprogramación de Deudas por Lluvias,Proyecto de Ley 4525 y 4704: Reprogramación de...,4704,12/04/2023,OLIVOS MARTÍNEZ VIVIAN,"Lizarzaburu Lizarzaburu, Juan Carlos Martin\nJ...",Fuerza Popular,EN COMISIÓN (Seguimientos),PROPONE ESTABLECER DISPOSICIONES EXTRAORDINARI...,https://wb2server.congreso.gob.pe/spley-portal...,3,2,2,3.0,NaN,Medio,89,0.042878,both
8,Compensación de Cuentas,Compensación en cuentas bancarias,4939,10/05/2023,YARROW LUMBRERAS NORMA MARTINA,"Chirinos Venegas, Patricia Rosa\nBazán Calderó...",Avanza País - Partido de Integración Social,EN COMISIÓN (Seguimientos),PROPONE MODIFICAR LA LEY N°29571 CÓDIGO DE PRO...,https://wb2server.congreso.gob.pe/spley-portal...,3,3,3,3.0,NaN,Alto,61,0.058308,both


In [ ]:
columns_to_mean = ['dimensión 01: Nivel de incremento de costos','dimensión 02: Afecta Estado de Derecho','dimensión 03: Nivel de aumento de Carga Operativa','dimensión 04: Disminución de ingresos','dimensión 05: Disminución del nivel de competitividad']

df['impacto'] = df[columns_to_mean].mean(axis=1).round(0)


In [ ]:
df.head()

,Nombre grupo,Nombre Corto,Numero,fechapresenta,autor,coautor,grupoparla,ultimoestado,sumilla,enlace,dimensión 01: Nivel de incremento de costos,dimensión 02: Afecta Estado de Derecho,dimensión 03: Nivel de aumento de Carga Operativa,dimensión 04: Disminución de ingresos,dimensión 05: Disminución del nivel de competitividad,Posibilidad de Aprobación,brecha_fecha,label_pred_prob,_merge,impacto
0,Comisiones Bancarias,Interplaza,2871,19/08/2022,SOTO PALACIOS WILSON,"Martínez Talavera, Pedro Edwin\nDoroteo Carbaj...",Acción Popular,EN COMISIÓN (Seguimientos),PROPONE PROTEGER AL CONSUMIDOR BANCARIO Y FINA...,https://wb2server.congreso.gob.pe/spley-portal...,3,3,0,3.0,0.0,Alto,67,0.344312,both,2.0
1,Comisiones Bancarias,"Interplaza, transferencia y pago con otra tarjeta",3739,05/12/2022,FLORES ANCACHI JORGE LUIS,"Vergara Mendoza, Elvis Hernán\nDoroteo Carbajo...",Acción Popular,Dictamen (Seguimientos),"PROPONE ELIMINAR LA COMISIÓN INTERPLAZA, LA CO...",https://wb2server.congreso.gob.pe/spley-portal...,3,3,3,NaN,NaN,Alto,32,0.110204,both,3.0
2,Comisiones Bancarias,Pago con cualquier tarjeta (débito o crédito),3262,10/10/2022,UGARTE MAMANI JHAKELINE KATY,"Quiroz Barboza, Segundo Teodomiro\nGutiérrez T...",Bloque Magisterial de Concertación Nacional,EN COMISIÓN (Seguimientos),PROPONE MODIFICAR EL CÓDIGO DE PROTECCIÓN Y DE...,https://wb2server.congreso.gob.pe/spley-portal...,3,3,3,NaN,NaN,Alto,271,0.107684,both,3.0
3,Comisiones Bancarias,Comisión por transferencia,3428,28/10/2022,BAZÁN NARRO SIGRID TESORO,"Luque Ibarra, Ruth\nCortez Aguirre, Isabel",Cambio Democrático - Juntos por el Perú,Dictamen (Seguimientos),PROPONE PROTEGER LOS DERECHOS DE LOS CONSUMIDO...,https://wb2server.congreso.gob.pe/spley-portal...,3,3,3,NaN,NaN,Alto,32,0.098295,both,3.0
4,Comisiones Bancarias,Comisión por depósito y retiro de dinero en ve...,4359,02/03/2023,CALLE LOBATÓN DIGNA,"Luna Gálvez, José León\nElías Ávalos, José Lui...",Podemos Perú,EN COMISIÓN (Seguimientos),PROPONE ELIMINAR LA COMISIÓN POR RETIRO DE DIN...,https://wb2server.congreso.gob.pe/spley-portal...,3,3,2,NaN,NaN,Alto,130,0.033485,both,3.0


In [ ]:
df.insert(15, 'impacto', df.pop('impacto'))

In [ ]:
df.head()

,Nombre grupo,Nombre Corto,Numero,fechapresenta,autor,coautor,grupoparla,ultimoestado,sumilla,enlace,dimensión 01: Nivel de incremento de costos,dimensión 02: Afecta Estado de Derecho,dimensión 03: Nivel de aumento de Carga Operativa,dimensión 04: Disminución de ingresos,dimensión 05: Disminución del nivel de competitividad,impacto,Posibilidad de Aprobación,brecha_fecha,label_pred_prob,_merge
0,Comisiones Bancarias,Interplaza,2871,19/08/2022,SOTO PALACIOS WILSON,"Martínez Talavera, Pedro Edwin\nDoroteo Carbaj...",Acción Popular,EN COMISIÓN (Seguimientos),PROPONE PROTEGER AL CONSUMIDOR BANCARIO Y FINA...,https://wb2server.congreso.gob.pe/spley-portal...,3,3,0,3.0,0.0,2.0,Alto,67,0.344312,both
1,Comisiones Bancarias,"Interplaza, transferencia y pago con otra tarjeta",3739,05/12/2022,FLORES ANCACHI JORGE LUIS,"Vergara Mendoza, Elvis Hernán\nDoroteo Carbajo...",Acción Popular,Dictamen (Seguimientos),"PROPONE ELIMINAR LA COMISIÓN INTERPLAZA, LA CO...",https://wb2server.congreso.gob.pe/spley-portal...,3,3,3,NaN,NaN,3.0,Alto,32,0.110204,both
2,Comisiones Bancarias,Pago con cualquier tarjeta (débito o crédito),3262,10/10/2022,UGARTE MAMANI JHAKELINE KATY,"Quiroz Barboza, Segundo Teodomiro\nGutiérrez T...",Bloque Magisterial de Concertación Nacional,EN COMISIÓN (Seguimientos),PROPONE MODIFICAR EL CÓDIGO DE PROTECCIÓN Y DE...,https://wb2server.congreso.gob.pe/spley-portal...,3,3,3,NaN,NaN,3.0,Alto,271,0.107684,both
3,Comisiones Bancarias,Comisión por transferencia,3428,28/10/2022,BAZÁN NARRO SIGRID TESORO,"Luque Ibarra, Ruth\nCortez Aguirre, Isabel",Cambio Democrático - Juntos por el Perú,Dictamen (Seguimientos),PROPONE PROTEGER LOS DERECHOS DE LOS CONSUMIDO...,https://wb2server.congreso.gob.pe/spley-portal...,3,3,3,NaN,NaN,3.0,Alto,32,0.098295,both
4,Comisiones Bancarias,Comisión por depósito y retiro de dinero en ve...,4359,02/03/2023,CALLE LOBATÓN DIGNA,"Luna Gálvez, José León\nElías Ávalos, José Lui...",Podemos Perú,EN COMISIÓN (Seguimientos),PROPONE ELIMINAR LA COMISIÓN POR RETIRO DE DIN...,https://wb2server.congreso.gob.pe/spley-portal...,3,3,2,NaN,NaN,3.0,Alto,130,0.033485,both


In [ ]:
df['_merge'].value_counts()

both          9
left_only     0
right_only    0
Name: _merge, dtype: int64

In [ ]:
df = df.drop(['_merge'], axis=1)

## 05 02. Cálculo de la posibilidad de aprobación"

In [ ]:
df['prob_aprob'] = df['label_pred_prob']

In [ ]:
df['label_pred_prob'].describe()

count    9.000000
mean     0.099995
std      0.096174
min      0.033485
25%      0.042878
50%      0.061979
75%      0.107684
max      0.344312
Name: label_pred_prob, dtype: float64

In [ ]:
std_deviation = df['label_pred_prob'].std()
std_deviation

0.09617406381418829

In [ ]:
# Esta variable ya es dada.
df['Posibilidad de Aprobación'].value_counts()

Alto     6
Medio    3
Name: Posibilidad de Aprobación, dtype: int64

In [ ]:
df.loc[df['Posibilidad de Aprobación'] == 'Alto', 'prob_aprob'] += df['label_pred_prob'].std()

In [ ]:
df.loc[df['Posibilidad de Aprobación'] == 'Medio', 'prob_aprob'] += std_deviation/2
df.loc[df['Posibilidad de Aprobación'] == 'Bajo', 'prob_aprob'] += std_deviation/4

In [ ]:
df

,Nombre grupo,Nombre Corto,Numero,fechapresenta,autor,coautor,grupoparla,ultimoestado,sumilla,enlace,dimensión 01: Nivel de incremento de costos,dimensión 02: Afecta Estado de Derecho,dimensión 03: Nivel de aumento de Carga Operativa,dimensión 04: Disminución de ingresos,dimensión 05: Disminución del nivel de competitividad,impacto,Posibilidad de Aprobación,brecha_fecha,label_pred_prob,prob_aprob
0,Comisiones Bancarias,Interplaza,2871,19/08/2022,SOTO PALACIOS WILSON,"Martínez Talavera, Pedro Edwin\nDoroteo Carbaj...",Acción Popular,EN COMISIÓN (Seguimientos),PROPONE PROTEGER AL CONSUMIDOR BANCARIO Y FINA...,https://wb2server.congreso.gob.pe/spley-portal...,3,3,0,3.0,0.0,2.0,Alto,67,0.344312,0.440486
1,Comisiones Bancarias,"Interplaza, transferencia y pago con otra tarjeta",3739,05/12/2022,FLORES ANCACHI JORGE LUIS,"Vergara Mendoza, Elvis Hernán\nDoroteo Carbajo...",Acción Popular,Dictamen (Seguimientos),"PROPONE ELIMINAR LA COMISIÓN INTERPLAZA, LA CO...",https://wb2server.congreso.gob.pe/spley-portal...,3,3,3,NaN,NaN,3.0,Alto,32,0.110204,0.206378
2,Comisiones Bancarias,Pago con cualquier tarjeta (débito o crédito),3262,10/10/2022,UGARTE MAMANI JHAKELINE KATY,"Quiroz Barboza, Segundo Teodomiro\nGutiérrez T...",Bloque Magisterial de Concertación Nacional,EN COMISIÓN (Seguimientos),PROPONE MODIFICAR EL CÓDIGO DE PROTECCIÓN Y DE...,https://wb2server.congreso.gob.pe/spley-portal...,3,3,3,NaN,NaN,3.0,Alto,271,0.107684,0.203858
3,Comisiones Bancarias,Comisión por transferencia,3428,28/10/2022,BAZÁN NARRO SIGRID TESORO,"Luque Ibarra, Ruth\nCortez Aguirre, Isabel",Cambio Democrático - Juntos por el Perú,Dictamen (Seguimientos),PROPONE PROTEGER LOS DERECHOS DE LOS CONSUMIDO...,https://wb2server.congreso.gob.pe/spley-portal...,3,3,3,NaN,NaN,3.0,Alto,32,0.098295,0.194469
4,Comisiones Bancarias,Comisión por depósito y retiro de dinero en ve...,4359,02/03/2023,CALLE LOBATÓN DIGNA,"Luna Gálvez, José León\nElías Ávalos, José Lui...",Podemos Perú,EN COMISIÓN (Seguimientos),PROPONE ELIMINAR LA COMISIÓN POR RETIRO DE DIN...,https://wb2server.congreso.gob.pe/spley-portal...,3,3,2,NaN,NaN,3.0,Alto,130,0.033485,0.129659
5,Comisiones Bancarias,Bloqueo de banca móvil,4360,02/03/2023,CALLE LOBATÓN DIGNA,"Luna Gálvez, José León\nElías Ávalos, José Lui...",Podemos Perú,EN COMISIÓN (Seguimientos),PROPONE RECONOCER EL DERECHO A LOS CONSUMIDORE...,https://wb2server.congreso.gob.pe/spley-portal...,3,2,2,NaN,NaN,2.0,Medio,130,0.061979,0.110066
6,Reprogramación de Deudas por Lluvias,Proyecto de Ley 4525 y 4704: Reprogramación de...,4525,21/03/2023,ALEGRÍA GARCÍA ARTURO,"Lizarzaburu Lizarzaburu, Juan Carlos Martin\nG...",Fuerza Popular,EN COMISIÓN (Seguimientos),PROPONE LEY DE CONGELAMIENTO DE DEUDAS DE PERS...,https://wb2server.congreso.gob.pe/spley-portal...,3,2,2,3.0,NaN,2.0,Medio,111,0.042814,0.090901
7,Reprogramación de Deudas por Lluvias,Proyecto de Ley 4525 y 4704: Reprogramación de...,4704,12/04/2023,OLIVOS MARTÍNEZ VIVIAN,"Lizarzaburu Lizarzaburu, Juan Carlos Martin\nJ...",Fuerza Popular,EN COMISIÓN (Seguimientos),PROPONE ESTABLECER DISPOSICIONES EXTRAORDINARI...,https://wb2server.congreso.gob.pe/spley-portal...,3,2,2,3.0,NaN,2.0,Medio,89,0.042878,0.090965
8,Compensación de Cuentas,Compensación en cuentas bancarias,4939,10/05/2023,YARROW LUMBRERAS NORMA MARTINA,"Chirinos Venegas, Patricia Rosa\nBazán Calderó...",Avanza País - Partido de Integración Social,EN COMISIÓN (Seguimientos),PROPONE MODIFICAR LA LEY N°29571 CÓDIGO DE PRO...,https://wb2server.congreso.gob.pe/spley-portal...,3,3,3,3.0,NaN,3.0,Alto,61,0.058308,0.154482


## 05 03. Cálculo Posibilidad de aprobación x Impacto

In [ ]:
df['indicador_final'] = df['prob_aprob'] * df['impacto']

In [ ]:
df

,Nombre grupo,Nombre Corto,Numero,fechapresenta,autor,coautor,grupoparla,ultimoestado,sumilla,enlace,dimensión 01: Nivel de incremento de costos,dimensión 02: Afecta Estado de Derecho,dimensión 03: Nivel de aumento de Carga Operativa,dimensión 04: Disminución de ingresos,dimensión 05: Disminución del nivel de competitividad,impacto,Posibilidad de Aprobación,brecha_fecha,label_pred_prob,prob_aprob,indicador_final
0,Comisiones Bancarias,Interplaza,2871,19/08/2022,SOTO PALACIOS WILSON,"Martínez Talavera, Pedro Edwin\nDoroteo Carbaj...",Acción Popular,EN COMISIÓN (Seguimientos),PROPONE PROTEGER AL CONSUMIDOR BANCARIO Y FINA...,https://wb2server.congreso.gob.pe/spley-portal...,3,3,0,3.0,0.0,2.0,Alto,67,0.344312,0.440486,0.880972
1,Comisiones Bancarias,"Interplaza, transferencia y pago con otra tarjeta",3739,05/12/2022,FLORES ANCACHI JORGE LUIS,"Vergara Mendoza, Elvis Hernán\nDoroteo Carbajo...",Acción Popular,Dictamen (Seguimientos),"PROPONE ELIMINAR LA COMISIÓN INTERPLAZA, LA CO...",https://wb2server.congreso.gob.pe/spley-portal...,3,3,3,NaN,NaN,3.0,Alto,32,0.110204,0.206378,0.619135
2,Comisiones Bancarias,Pago con cualquier tarjeta (débito o crédito),3262,10/10/2022,UGARTE MAMANI JHAKELINE KATY,"Quiroz Barboza, Segundo Teodomiro\nGutiérrez T...",Bloque Magisterial de Concertación Nacional,EN COMISIÓN (Seguimientos),PROPONE MODIFICAR EL CÓDIGO DE PROTECCIÓN Y DE...,https://wb2server.congreso.gob.pe/spley-portal...,3,3,3,NaN,NaN,3.0,Alto,271,0.107684,0.203858,0.611573
3,Comisiones Bancarias,Comisión por transferencia,3428,28/10/2022,BAZÁN NARRO SIGRID TESORO,"Luque Ibarra, Ruth\nCortez Aguirre, Isabel",Cambio Democrático - Juntos por el Perú,Dictamen (Seguimientos),PROPONE PROTEGER LOS DERECHOS DE LOS CONSUMIDO...,https://wb2server.congreso.gob.pe/spley-portal...,3,3,3,NaN,NaN,3.0,Alto,32,0.098295,0.194469,0.583407
4,Comisiones Bancarias,Comisión por depósito y retiro de dinero en ve...,4359,02/03/2023,CALLE LOBATÓN DIGNA,"Luna Gálvez, José León\nElías Ávalos, José Lui...",Podemos Perú,EN COMISIÓN (Seguimientos),PROPONE ELIMINAR LA COMISIÓN POR RETIRO DE DIN...,https://wb2server.congreso.gob.pe/spley-portal...,3,3,2,NaN,NaN,3.0,Alto,130,0.033485,0.129659,0.388977
5,Comisiones Bancarias,Bloqueo de banca móvil,4360,02/03/2023,CALLE LOBATÓN DIGNA,"Luna Gálvez, José León\nElías Ávalos, José Lui...",Podemos Perú,EN COMISIÓN (Seguimientos),PROPONE RECONOCER EL DERECHO A LOS CONSUMIDORE...,https://wb2server.congreso.gob.pe/spley-portal...,3,2,2,NaN,NaN,2.0,Medio,130,0.061979,0.110066,0.220132
6,Reprogramación de Deudas por Lluvias,Proyecto de Ley 4525 y 4704: Reprogramación de...,4525,21/03/2023,ALEGRÍA GARCÍA ARTURO,"Lizarzaburu Lizarzaburu, Juan Carlos Martin\nG...",Fuerza Popular,EN COMISIÓN (Seguimientos),PROPONE LEY DE CONGELAMIENTO DE DEUDAS DE PERS...,https://wb2server.congreso.gob.pe/spley-portal...,3,2,2,3.0,NaN,2.0,Medio,111,0.042814,0.090901,0.181801
7,Reprogramación de Deudas por Lluvias,Proyecto de Ley 4525 y 4704: Reprogramación de...,4704,12/04/2023,OLIVOS MARTÍNEZ VIVIAN,"Lizarzaburu Lizarzaburu, Juan Carlos Martin\nJ...",Fuerza Popular,EN COMISIÓN (Seguimientos),PROPONE ESTABLECER DISPOSICIONES EXTRAORDINARI...,https://wb2server.congreso.gob.pe/spley-portal...,3,2,2,3.0,NaN,2.0,Medio,89,0.042878,0.090965,0.181930
8,Compensación de Cuentas,Compensación en cuentas bancarias,4939,10/05/2023,YARROW LUMBRERAS NORMA MARTINA,"Chirinos Venegas, Patricia Rosa\nBazán Calderó...",Avanza País - Partido de Integración Social,EN COMISIÓN (Seguimientos),PROPONE MODIFICAR LA LEY N°29571 CÓDIGO DE PRO...,https://wb2server.congreso.gob.pe/spley-portal...,3,3,3,3.0,NaN,3.0,Alto,61,0.058308,0.154482,0.463445


In [ ]:
df['indicador_final'].describe()

count    9.000000
mean     0.459041
std      0.239282
min      0.181801
25%      0.220132
50%      0.463445
75%      0.611573
max      0.880972
Name: indicador_final, dtype: float64

In [ ]:
lower_tercile = np.percentile(df['indicador_final'], 33)
upper_tercile = np.percentile(df['indicador_final'], 66)

In [ ]:
# Convert to categorical variable
df['indicador_final_nivel'] = pd.cut(df['indicador_final'],
                                   bins=[float('-inf'), lower_tercile, upper_tercile, float('inf')],
                                   labels=['BAJO', 'MEDIO', 'ALTO'])

In [ ]:
df

,Nombre grupo,Nombre Corto,Numero,fechapresenta,autor,coautor,grupoparla,ultimoestado,sumilla,enlace,dimensión 01: Nivel de incremento de costos,dimensión 02: Afecta Estado de Derecho,dimensión 03: Nivel de aumento de Carga Operativa,dimensión 04: Disminución de ingresos,dimensión 05: Disminución del nivel de competitividad,impacto,Posibilidad de Aprobación,brecha_fecha,label_pred_prob,prob_aprob,indicador_final,indicador_final_nivel
0,Comisiones Bancarias,Interplaza,2871,19/08/2022,SOTO PALACIOS WILSON,"Martínez Talavera, Pedro Edwin\nDoroteo Carbaj...",Acción Popular,EN COMISIÓN (Seguimientos),PROPONE PROTEGER AL CONSUMIDOR BANCARIO Y FINA...,https://wb2server.congreso.gob.pe/spley-portal...,3,3,0,3.0,0.0,2.0,Alto,67,0.344312,0.440486,0.880972,ALTO
1,Comisiones Bancarias,"Interplaza, transferencia y pago con otra tarjeta",3739,05/12/2022,FLORES ANCACHI JORGE LUIS,"Vergara Mendoza, Elvis Hernán\nDoroteo Carbajo...",Acción Popular,Dictamen (Seguimientos),"PROPONE ELIMINAR LA COMISIÓN INTERPLAZA, LA CO...",https://wb2server.congreso.gob.pe/spley-portal...,3,3,3,NaN,NaN,3.0,Alto,32,0.110204,0.206378,0.619135,ALTO
2,Comisiones Bancarias,Pago con cualquier tarjeta (débito o crédito),3262,10/10/2022,UGARTE MAMANI JHAKELINE KATY,"Quiroz Barboza, Segundo Teodomiro\nGutiérrez T...",Bloque Magisterial de Concertación Nacional,EN COMISIÓN (Seguimientos),PROPONE MODIFICAR EL CÓDIGO DE PROTECCIÓN Y DE...,https://wb2server.congreso.gob.pe/spley-portal...,3,3,3,NaN,NaN,3.0,Alto,271,0.107684,0.203858,0.611573,ALTO
3,Comisiones Bancarias,Comisión por transferencia,3428,28/10/2022,BAZÁN NARRO SIGRID TESORO,"Luque Ibarra, Ruth\nCortez Aguirre, Isabel",Cambio Democrático - Juntos por el Perú,Dictamen (Seguimientos),PROPONE PROTEGER LOS DERECHOS DE LOS CONSUMIDO...,https://wb2server.congreso.gob.pe/spley-portal...,3,3,3,NaN,NaN,3.0,Alto,32,0.098295,0.194469,0.583407,MEDIO
4,Comisiones Bancarias,Comisión por depósito y retiro de dinero en ve...,4359,02/03/2023,CALLE LOBATÓN DIGNA,"Luna Gálvez, José León\nElías Ávalos, José Lui...",Podemos Perú,EN COMISIÓN (Seguimientos),PROPONE ELIMINAR LA COMISIÓN POR RETIRO DE DIN...,https://wb2server.congreso.gob.pe/spley-portal...,3,3,2,NaN,NaN,3.0,Alto,130,0.033485,0.129659,0.388977,MEDIO
5,Comisiones Bancarias,Bloqueo de banca móvil,4360,02/03/2023,CALLE LOBATÓN DIGNA,"Luna Gálvez, José León\nElías Ávalos, José Lui...",Podemos Perú,EN COMISIÓN (Seguimientos),PROPONE RECONOCER EL DERECHO A LOS CONSUMIDORE...,https://wb2server.congreso.gob.pe/spley-portal...,3,2,2,NaN,NaN,2.0,Medio,130,0.061979,0.110066,0.220132,BAJO
6,Reprogramación de Deudas por Lluvias,Proyecto de Ley 4525 y 4704: Reprogramación de...,4525,21/03/2023,ALEGRÍA GARCÍA ARTURO,"Lizarzaburu Lizarzaburu, Juan Carlos Martin\nG...",Fuerza Popular,EN COMISIÓN (Seguimientos),PROPONE LEY DE CONGELAMIENTO DE DEUDAS DE PERS...,https://wb2server.congreso.gob.pe/spley-portal...,3,2,2,3.0,NaN,2.0,Medio,111,0.042814,0.090901,0.181801,BAJO
7,Reprogramación de Deudas por Lluvias,Proyecto de Ley 4525 y 4704: Reprogramación de...,4704,12/04/2023,OLIVOS MARTÍNEZ VIVIAN,"Lizarzaburu Lizarzaburu, Juan Carlos Martin\nJ...",Fuerza Popular,EN COMISIÓN (Seguimientos),PROPONE ESTABLECER DISPOSICIONES EXTRAORDINARI...,https://wb2server.congreso.gob.pe/spley-portal...,3,2,2,3.0,NaN,2.0,Medio,89,0.042878,0.090965,0.181930,BAJO
8,Compensación de Cuentas,Compensación en cuentas bancarias,4939,10/05/2023,YARROW LUMBRERAS NORMA MARTINA,"Chirinos Venegas, Patricia Rosa\nBazán Calderó...",Avanza País - Partido de Integración Social,EN COMISIÓN (Seguimientos),PROPONE MODIFICAR LA LEY N°29571 CÓDIGO DE PRO...,https://wb2server.congreso.gob.pe/spley-portal...,3,3,3,3.0,NaN,3.0,Alto,61,0.058308,0.154482,0.463445,MEDIO


## 05 04. Creación columna fecha hoy

In [ ]:
df['fecha_scrapping'] = datetime.now()
#df['fecha_scrapping'] = pd.to_datetime(df['fecha_scrapping'], format='%d/%m/%Y')
df['fecha_scrapping'] = df['fecha_scrapping'].dt.strftime('%d-%m-%Y')
df['fecha_scrapping']

0    12-07-2023
1    12-07-2023
2    12-07-2023
3    12-07-2023
4    12-07-2023
5    12-07-2023
6    12-07-2023
7    12-07-2023
8    12-07-2023
Name: fecha_scrapping, dtype: object

In [ ]:
df.replace(np.nan, '', inplace = True)

# 06 Exportación de base parcial

In [ ]:
df.columns

Index(['Nombre grupo', 'Nombre Corto', 'Numero', 'fechapresenta', 'autor',
       'coautor', 'grupoparla', 'ultimoestado', 'sumilla', 'enlace',
       'dimensión 01: Nivel de incremento de costos',
       'dimensión 02: Afecta Estado de Derecho',
       'dimensión 03: Nivel de aumento de Carga Operativa',
       'dimensión 04: Disminución de ingresos',
       'dimensión 05: Disminución del nivel de competitividad', 'impacto',
       'Posibilidad de Aprobación', 'brecha_fecha', 'label_pred_prob',
       'prob_aprob', 'indicador_final', 'indicador_final_nivel',
       'fecha_scrapping'],
      dtype='object')

In [ ]:
lin = 'https://docs.google.com/spreadsheets/d/1K63e7VtoTFWyccsQe276_ovoGM7249lYYFC6IL_T_VU/edit#gid=0'

sh2 = gc.open_by_url(lin)
wks1 = sh2[0]
wks1.clear(start='A1', end=None)
wks1.set_dataframe(df, 'A1')

# 07 Exportación de base append

In [ ]:
df_append = pd.concat([append_output,df],axis=0)

In [ ]:
df_append

,Nombre grupo,Nombre Corto,Numero,fechapresenta,autor,enlace,dimensión 01: Nivel de incremento de costos,dimensión 02: Afecta Estado de Derecho,dimesión 03: Nivel de aumento de Carga Operativa,dimensión 04: Disminución de ingresos,dimensión 05: Disminución del nivel de competitividad,impacto,Posibilidad de Aprobación,ultimoestado,brecha_fecha,label_pred_prob,prob_aprob,indicador_final,indicador_final_nivel,fecha_scrapping,coautor,grupoparla,comisiones
0,Comisiones Bancarias,Interplaza,2871,19/08/2022,SOTO PALACIOS WILSON,https://wb2server.congreso.gob.pe/spley-portal...,3,3,0,3,0,2.0,Alto,EN COMISIÓN (Seguimientos),52,0.344312,0.440486,0.880972,ALTO,27-06-2023,,,
1,Comisiones Bancarias,"Interplaza, transferencia y pago con otra tarjeta",3739,05/12/2022,FLORES ANCACHI JORGE LUIS,https://wb2server.congreso.gob.pe/spley-portal...,3,3,3,,,3.0,Alto,Dictamen (Seguimientos),17,0.110204,0.206378,0.619135,ALTO,27-06-2023,,,
2,Comisiones Bancarias,Pago con cualquier tarjeta (débito o crédito),3262,10/10/2022,UGARTE MAMANI JHAKELINE KATY,https://wb2server.congreso.gob.pe/spley-portal...,3,3,3,,,3.0,Alto,EN COMISIÓN (Seguimientos),256,0.107684,0.203858,0.611573,ALTO,27-06-2023,,,
3,Comisiones Bancarias,Comisión por transferencia,3428,28/10/2022,BAZÁN NARRO SIGRID TESORO,https://wb2server.congreso.gob.pe/spley-portal...,3,3,3,,,3.0,Alto,Dictamen (Seguimientos),17,0.098295,0.194469,0.583407,MEDIO,27-06-2023,,,
4,Comisiones Bancarias,Comisión por depósito y retiro de dinero en ve...,4359,02/03/2023,CALLE LOBATÓN DIGNA,https://wb2server.congreso.gob.pe/spley-portal...,3,3,2,,,3.0,Alto,EN COMISIÓN (Seguimientos),115,0.033485,0.129659,0.388977,MEDIO,27-06-2023,,,
5,Comisiones Bancarias,Bloqueo de banca móvil,4360,02/03/2023,CALLE LOBATÓN DIGNA,https://wb2server.congreso.gob.pe/spley-portal...,3,2,2,,,2.0,Medio,EN COMISIÓN (Seguimientos),115,0.061979,0.110066,0.220132,BAJO,27-06-2023,,,
6,Reprogramación de Deudas por Lluvias,Proyecto de Ley 4525 y 4704: Reprogramación de...,4525,21/03/2023,ALEGRÍA GARCÍA ARTURO,https://wb2server.congreso.gob.pe/spley-portal...,3,2,2,3,,2.0,Medio,EN COMISIÓN (Seguimientos),96,0.042814,0.090901,0.181801,BAJO,27-06-2023,,,
7,Reprogramación de Deudas por Lluvias,Proyecto de Ley 4525 y 4704: Reprogramación de...,4704,12/04/2023,OLIVOS MARTÍNEZ VIVIAN,https://wb2server.congreso.gob.pe/spley-portal...,3,2,2,3,,2.0,Medio,EN COMISIÓN (Seguimientos),74,0.042878,0.090965,0.181930,BAJO,27-06-2023,,,
8,Compensación de Cuentas,Compensación en cuentas bancarias,4939,10/05/2023,YARROW LUMBRERAS NORMA MARTINA,https://wb2server.congreso.gob.pe/spley-portal...,3,3,3,3,,3.0,Alto,EN COMISIÓN (Seguimientos),46,0.058308,0.154482,0.463445,MEDIO,27-06-2023,,,
9,Comisiones Bancarias,Interplaza,2871,19/08/2022,SOTO PALACIOS WILSON,https://wb2server.congreso.gob.pe/spley-portal...,3,3,0,3,0,2.0,Alto,EN COMISIÓN (Seguimientos),66,0.344312,0.440486,0.880972,ALTO,10-07-2023,,,


In [ ]:
df_append.replace(np.nan, '', inplace = True)
df_append

,Nombre grupo,Nombre Corto,Numero,fechapresenta,autor,enlace,dimensión 01: Nivel de incremento de costos,dimensión 02: Afecta Estado de Derecho,dimesión 03: Nivel de aumento de Carga Operativa,dimensión 04: Disminución de ingresos,dimensión 05: Disminución del nivel de competitividad,impacto,Posibilidad de Aprobación,ultimoestado,brecha_fecha,label_pred_prob,prob_aprob,indicador_final,indicador_final_nivel,fecha_scrapping,coautor,grupoparla,comisiones
0,Comisiones Bancarias,Interplaza,2871,19/08/2022,SOTO PALACIOS WILSON,https://wb2server.congreso.gob.pe/spley-portal...,3,3,0,3,0,2.0,Alto,EN COMISIÓN (Seguimientos),52,0.344312,0.440486,0.880972,ALTO,27-06-2023,,,
1,Comisiones Bancarias,"Interplaza, transferencia y pago con otra tarjeta",3739,05/12/2022,FLORES ANCACHI JORGE LUIS,https://wb2server.congreso.gob.pe/spley-portal...,3,3,3,,,3.0,Alto,Dictamen (Seguimientos),17,0.110204,0.206378,0.619135,ALTO,27-06-2023,,,
2,Comisiones Bancarias,Pago con cualquier tarjeta (débito o crédito),3262,10/10/2022,UGARTE MAMANI JHAKELINE KATY,https://wb2server.congreso.gob.pe/spley-portal...,3,3,3,,,3.0,Alto,EN COMISIÓN (Seguimientos),256,0.107684,0.203858,0.611573,ALTO,27-06-2023,,,
3,Comisiones Bancarias,Comisión por transferencia,3428,28/10/2022,BAZÁN NARRO SIGRID TESORO,https://wb2server.congreso.gob.pe/spley-portal...,3,3,3,,,3.0,Alto,Dictamen (Seguimientos),17,0.098295,0.194469,0.583407,MEDIO,27-06-2023,,,
4,Comisiones Bancarias,Comisión por depósito y retiro de dinero en ve...,4359,02/03/2023,CALLE LOBATÓN DIGNA,https://wb2server.congreso.gob.pe/spley-portal...,3,3,2,,,3.0,Alto,EN COMISIÓN (Seguimientos),115,0.033485,0.129659,0.388977,MEDIO,27-06-2023,,,
5,Comisiones Bancarias,Bloqueo de banca móvil,4360,02/03/2023,CALLE LOBATÓN DIGNA,https://wb2server.congreso.gob.pe/spley-portal...,3,2,2,,,2.0,Medio,EN COMISIÓN (Seguimientos),115,0.061979,0.110066,0.220132,BAJO,27-06-2023,,,
6,Reprogramación de Deudas por Lluvias,Proyecto de Ley 4525 y 4704: Reprogramación de...,4525,21/03/2023,ALEGRÍA GARCÍA ARTURO,https://wb2server.congreso.gob.pe/spley-portal...,3,2,2,3,,2.0,Medio,EN COMISIÓN (Seguimientos),96,0.042814,0.090901,0.181801,BAJO,27-06-2023,,,
7,Reprogramación de Deudas por Lluvias,Proyecto de Ley 4525 y 4704: Reprogramación de...,4704,12/04/2023,OLIVOS MARTÍNEZ VIVIAN,https://wb2server.congreso.gob.pe/spley-portal...,3,2,2,3,,2.0,Medio,EN COMISIÓN (Seguimientos),74,0.042878,0.090965,0.181930,BAJO,27-06-2023,,,
8,Compensación de Cuentas,Compensación en cuentas bancarias,4939,10/05/2023,YARROW LUMBRERAS NORMA MARTINA,https://wb2server.congreso.gob.pe/spley-portal...,3,3,3,3,,3.0,Alto,EN COMISIÓN (Seguimientos),46,0.058308,0.154482,0.463445,MEDIO,27-06-2023,,,
9,Comisiones Bancarias,Interplaza,2871,19/08/2022,SOTO PALACIOS WILSON,https://wb2server.congreso.gob.pe/spley-portal...,3,3,0,3,0,2.0,Alto,EN COMISIÓN (Seguimientos),66,0.344312,0.440486,0.880972,ALTO,10-07-2023,,,


In [ ]:
#Exportación de la base:

wks1 = sh2[1]
wks1.clear(start='A1', end=None)
wks1.set_dataframe(df_append, 'A1')

# 08 Exportación "seguimientocongreso"

In [ ]:
df_s=pd.merge(evaluacion[['Nombre grupo','Nombre Corto','Numero','enlace']], scrapping_seguimiento[['FECHA','ESTADO PROCESAL','enlace','DETALLE','COMISIÓN']], on="enlace", how="inner", indicator=True)
df_s.head(2)

,Nombre grupo,Nombre Corto,Numero,enlace,FECHA,ESTADO PROCESAL,DETALLE,COMISIÓN,_merge
0,Comisiones Bancarias,Interplaza,2871,https://wb2server.congreso.gob.pe/spley-portal...,2022-08-19,PRESENTADO,LEY QUE PROTEGE AL CONSUMIDOR BANCARIO Y FINAN...,,both
1,Comisiones Bancarias,Interplaza,2871,https://wb2server.congreso.gob.pe/spley-portal...,2022-08-24,EN COMISIÓN,,Defensa del Consumidor y Organismos Reguladore...,both


In [ ]:
df_s.insert(7, 'enlace', df_s.pop('enlace'))


In [ ]:
df_s = df_s.drop(['_merge'], axis=1)

In [ ]:
df_s['FECHA'] = pd.to_datetime(df_s['FECHA'])
df_s['FECHA'] = df_s['FECHA'].dt.strftime('%d-%m-%Y')
df_s.head()

,Nombre grupo,Nombre Corto,Numero,FECHA,ESTADO PROCESAL,DETALLE,COMISIÓN,enlace
0,Comisiones Bancarias,Interplaza,2871,19-08-2022,PRESENTADO,LEY QUE PROTEGE AL CONSUMIDOR BANCARIO Y FINAN...,,https://wb2server.congreso.gob.pe/spley-portal...
1,Comisiones Bancarias,Interplaza,2871,24-08-2022,EN COMISIÓN,,Defensa del Consumidor y Organismos Reguladore...,https://wb2server.congreso.gob.pe/spley-portal...
2,Comisiones Bancarias,Interplaza,2871,27-04-2023,Dictamen,EN MAYORÍA - FÓRMULA SUSTITUTORIA - LEY QUE MO...,Defensa del Consumidor y Organismos Reguladore...,https://wb2server.congreso.gob.pe/spley-portal...
3,Comisiones Bancarias,Interplaza,2871,05-05-2023,EN COMISIÓN,"EL CONSEJO DIRECTIVO, EN SU SESIÓN DEL 2 DE MA...","Economía, Banca, Finanzas e Inteligencia Finan...",https://wb2server.congreso.gob.pe/spley-portal...
4,Comisiones Bancarias,"Interplaza, transferencia y pago con otra tarjeta",3739,05-12-2022,PRESENTADO,"LEY QUE ELIMINA LA COMISIÓN INTERPLAZA, LA COM...",,https://wb2server.congreso.gob.pe/spley-portal...


In [ ]:
#Exportación de la base:

wks1 = sh2[2]
wks1.clear(start='A1', end=None)
wks1.set_dataframe(df_s, 'A1')

In [ ]:
# Término del script